In [18]:
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import matplotlib.dates as dates
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats

import os,sys
import glob
from matplotlib.dates import date2num
import seaborn as sns
import datetime as datetime
from copy import copy

from xmca.xarray import xMCA  # use with xr.DataArray
import dask
import toolz
import xarray as xr
%matplotlib qt5
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import explained_variance_score,mean_absolute_error
from sklearn.metrics import mean_squared_log_error,mean_squared_error,median_absolute_error

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import pymannkendall as mk

dask.config.set({"array.slicing.split_large_chunks": True})

In [118]:
plt.rcParams.update({'font.size': 18,'font.family':'serif','font.serif':'Arial'})
months=['J','F','M','A','M','J','J','A','S','O','N','D']

# Spectral analysis function

In [111]:
def specs(xxx,ppp,dt,win,alpha,smo,ci):
    
    """
% |  0) CALLS                                                                 
% | 
% | [hepya,fff,conflim] = specs(xxx,ppp,dt,win,alpha,smo,ci);
% |                                                                  
% | 1) INPUT / OUTPUT
% |    xxx = time series for spectral analysis
% |    ppp = number of points for analysis
% |    dt  = sampling interval
% |    win = time window type (0, 1 or 2)
% |    alpha = (0 >= alpha <= 1) Shape parameter of the Tukey window (win = 1), representing the fraction of 
% |            the window inside the cosine tapered region. 
% |            If zero,the Tukey window is equivalent to a rectangular window. 
% |            If one, the Tukey window is equivalent to a Hanning window.
% |    smo = control spectra smoothing
% |    ci  = confidence level
% | 
% |    hepya  = spectrum of time series for specs
% |    fff    = frequencies
% |    conflim = confidence limits for spectrum
% |
% | 2) TIME WINDOW
% |    Applyed in time domain to reduce syde lobs leakage. 
% |    Options are: win=0 (no window); 
% |                 win=1 (Hanning window) and 
% |                 win=2 (cossine tappered window) 
% |    For win=2, size of window can be controlled editing value of wid or alpha.
% |    Default is 10% (wid=0.1)
% |
% | 3) SPECTRUM
% |    Computed using FFT of whole series. Can have number of points
% |    for analysis (ppp) as the same of time series, or increase to
% |    next power of 2 (fill with zeros) or any value.
% |    Computed using numpy.fft.fft, similar to MATLAB fft.  
% |    Spectrum unit is: 
% |       [variable unit square divided by cicles per time unit]
% |    Final variable is hepya, which length is (ppp/2+1)
% |
% | 4) TIME WINDOW CORRECTION
% |    Applied to spectrum and cross spectrum to correct for energy
% |    loss due to time window: 1.14 for cossine tappered and 2.6 for Hanning
% |
% | 5) SMOOTHING
% |    Running average smoothing of spectra to reduce variance and
% |    increase statistical confidence, or reduce confidence limits.
% |    It uses a Hamm window - a box window would introduce spurious 
% |    picks and phase problems.
% |    Get to compromising between strong smothing (more confidence
% |    but stronger bias) and weak smoothing (less confidence but
% |    less bias) - See Ref.b
% |    Two options are avaiable:
% |    a) Fixed window size (smo>1)
% |    b) Variable window size (smo=999). 
% |       The idea is to have smaller size for low frequencies,
% |    where have less points, and progressively increase size to 
% |    higher frequencies. Will have weaker smoothing for lower
% |    frequencies, with less statistical confidence, but will not
% |    lose information.
% |    If smo=999, than the width (smo=2*smo1+1) is increased by 
% |    (inc) at every interval (int). Default values are
% |    smo1=4; int=10; inc=2;                                     
% |    To control these values edit this line inside the program.
% |
% | 6) RESOLUTION
% |    Raw spectrum has resolution of B=1/T. Smoothed spectrum has
% |    resolution B=smo/T.
% |    Note that although B increases, program compute spectrum for
% |    all the frequencies of raw spectrum.
% | 
% | 7) DEGREES OF FREEDOM
% |    Used in computation of confident limits. Most simple
% |    definition is: 
% |       df=2*smo
% |    Here I use the one used by Rainer Zanttopp, where:
% |       df = 2 * smo * tap * wffac
% |    with (wffac=0.63) a correction for the effective size of the
% |    Hamming window and (tap=1-2*wid) a correction for the time 
% |    windowing with cossine tappered for first and last 100*wid%
% |    of the data. For Hanning window I used (tap=1) as I just do
% |    not know which correction to apply.
% |    See references a, e h, i
% | 
% | 
% | 9) CONFIDENCE LIMITS
% |    For spectrum, given by (ref. d , pg 286):
% |         {df/[Chi(df,a/2)]} <= s2 <= {df/[Chi(df,1-a/2)]}
% |    where Chi is Chi square distribution, df is number of degrees
% |    freedom and (a=1-p/100) for p=confidence level (e.g., 95%, 90%).
% | 
% | 
% | 11)PLOTS
% | 
% |    a) Spectra
% |       plot(fff,hepya)
% |       semilogx(fff,hepya)
% |       loglog(fff,hepya)
% |
% |    b) Confidence limits
% |       semilogx(fff,chi,fff,clo)
% |       loglog(fff,med*sc,fff,chi1*sc,fff,clo1*sc)
% |           where sc is ordinate of med
% | 
% |    c) confidence interval in loglog
% |       sc=0.01;chi1=sc*conflim(:,3);clo1=sc*conflim(:,4);
% |       f1=1;loglog(fff,hepya),hold on,line([f1 f1],[clo1 chi1])
% |       line([f1-0.001 f1+0.001],[chi1 chi1])
% |       line([f1-0.001 f1+0.001],[clo1 clo1])  % try values added to f1
% |       obs: f1 draws lines in x (abscissas) e sc draws lines in y (ordinates)
% |
% | 12)REFERENCES
% | 
% |    a) My notes
% |    
% |    b) Mello Filho, E. (1982). Investigacao sobre a analise da
% |       agitacao maritima, Theses, COPPE-UFRJ.
% |    
% |    c) Processamento e analise de sinais. Relatorio, PETROBRAS.
% |    
% |    d) Bendat, J.S and A.G. Piersol (1986). Random data - analysis
% |       and measurement procedures. John Willey & Sons.
% |    
% |    e) Jenkins, G.M and D.G. Watts (1968). Spectral analysis and
% |       its applications, Holden-day, S. Francisco.
% |    
% |    f) Julian, P.R. (1975). Comments on the determination of 
% |       significance level of the coherence statistics, Journal
% |       of the Atmospheric Sciences, 32, 836-837.
% |    
% |    g) Thompson, R.O.R.Y (1979) Coherence significance levels.
% |       Journal of the Atmospheric Sciences, october, 2020-2021.
% |    
% |    h) Cooley, Lewis and Welch (1967) The FFT algorithm and its
% |       applications, DOC RC1743, IBM Research, (pg 139).
% |    
% |    i) NCAR routine specft.for
% |       Emery and Thomson, Data Analysis in Oceanography, 
% |       chapter 5 (5.6.4.2)
% |    
    """
    
#    del(hwin,hxxx,hyyy,hepy,hepya,fff,chi,clo)
    
    # ********************  CREATE SPECTRAL WINDOW  ********************

    # -------------- length of window
    
    length = len(xxx)
    if length > ppp:
        winsize = ppp
    elif length <= ppp:
        winsize = length
        
    # --------------- no window
    if win == 0:
        hwin= np.ones(winsize)

    # --------------- Hanning window
    elif win == 1:
        hwin = np.hanning(winsize)

    # --------------- cosine-tapered applied to first and last 100*wid% of data
#    elif win == 2:
#        pi  = np.pi
#        wid = 0.1 # default
#        ii=np.arange(0,winsize)
#        hwin=np.ndarray(winsize)
#        
#        for jj in ii:
#            if jj <= wid*winsize: 
#                hwin[jj]=0.5 * (1-np.cos(5*pi*jj/winsize)) 
#        
#       
#            elif jj >= (winsize-wid*winsize+1):
#                hwin[jj] = 0.5 * (1+cos(5*pi*(jj-1)/winsize))
#        
#            else:
#                hwin[jj] = 1
#                
    elif win==2:
        
        hwin = sp.signal.windows.tukey(winsize,alpha) # apply tukey (cosine-tappered) window
                
    # ************ MAKE SURE ALL VECTORS ARE COLUMN VECTORS *************       
    # NOT NECESSARY IN NUMPY ARRAYS - IN CASE OF DOT PRODUCTS, PYTHON UNDERSTANDS USING NUMPY.DOT   
    # xxx = xxx(:)
    # hwin = hwin(:)

    # ********************  CALCULATE SPECTRUM USING FFT  ********************

    # --------------------  remove average
    xxx = xxx - np.mean(xxx)
 
    # --------------------  apply window
    hxxx=np.ndarray(winsize)
    hxxx = hwin[0:winsize]*xxx[0:winsize];

    # -------------------- get number of harmonics
    if (ppp%2)==1:         # ppp is odd
        nhar = (ppp+1)/2
    else:                  # ppp is even
        nhar = (ppp/2)+1
        
    nhar=int(nhar)

    # --------------------  calculate spectrum
    hyyy = np.fft.fft(hxxx,n=ppp)
    hepy = np.real(hyyy*np.conj(hyyy)/len(hyyy) * dt)
    hepy = hepy[:nhar]
    hepy[1:] = 2*hepy[1:];

    
    # --------------------  Window correction
    if win == 1:                           
        hepy[1:] = 2.6*hepy[1:]
        
    elif win == 2:
        hepy[1:] = 1.14*hepy[1:]
        

    # **************************  SMOOTH SPECTRUM  ***********************

    # --------------------  no smoothging

    if smo == 1:

        hepya = hepy.copy()

    # --------------------  variable smoothing weights
    elif smo == 999:
                
        # smo1=2; int=10; inc=1;
        smo1=4; intg=10; inc=2

        smo1a=smo1; ind=smo1; int1=intg
        
        hepya=np.ndarray(nhar-smo1)
        
        ii = smo1
        
        while ii+1 <= nhar-smo1:
            
            aux1 = sum(hepy[ii-smo1:ii+smo1+1]*np.hamming(2*smo1+1))
            aux2 = sum(np.hamming(2*smo1+1))
            hepya[ii-ind] = aux1 / aux2;
            flag=0;
            
            if ii+1 >= int1:
                smo1 = smo1 + inc;
                int1 = int1 + intg;
                flag=1;
                                           
            ii = ii + 1
                     
        if flag == 1: 
            smo1=smo1-inc 
        
        hepya=hepya[:2044]
        
    # --------------------  constant smoothing weights

    else:

        smo1 = (smo-1)/2
        hepya=np.ndarray(nhar-smo+1)
        for ii in range(0,nhar-smo+1):
            hepya[ii] = sum(hepy[ii:smo+ii]*np.hamming(smo))/(sum(np.hamming(smo)))

# ****************** CALCULATE CONFIDENCE INTERVAL *******************

# -------------  get time window factor for degrees of freedom
    if win == 0:
          wtfac = 1
    elif win == 1:
          wtfac = 1
    elif win == 2:
          wtfac = 1 - 2*wid

# -----------  width correction factor for Hamming freq. smoothing
    wffac = 0.63;

    alpha = 1 - ci/100;

    if smo == 999:

        x1=0 
        x2=intg 
        aux=smo1a
    
        chi=[]
        clo=[]
    
        while x1 < len(hepya):
#   df = 2 * (2*aux+1);
            df = round( 2 * (2*aux+1) * wtfac * wffac)
            chi_id = df * hepya[x1:x2] / sp.stats.distributions.chi2.ppf(alpha/2,df=df)
            chi.extend(chi_id)
            clo_id = df * hepya[x1:x2] / sp.stats.distributions.chi2.ppf(1-alpha/2,df=df)
            clo.extend(clo_id)
            x1=x2 
            x2=x2+intg+1 
            aux=aux+inc
        
            if x2 > len(hepya):
                x2=len(hepya)
    
        chi=np.asarray(chi)        
        clo=np.asarray(clo)        
        chi1 = (chi)/hepya
        clo1 = (clo)/hepya

    else:

#   df = 2 * (2*aux+1);
        df = round( 2 * smo * wtfac * wffac )
        chi = df * hepya / sp.stats.distributions.chi2.ppf(alpha/2,df=df)
        clo = df * hepya / sp.stats.distributions.chi2.ppf(1-alpha/2,df=df)
        chi1 = chi/hepya
        clo1 = clo/hepya

    med = np.ones(len(hepya))
    conflim=pd.DataFrame()
    conflim['chi']= chi
    conflim['clo']= clo
    conflim['chi1']= chi1
    conflim['clo1']= clo1
    conflim['med']= med
    
    # ***************  CALCULATE FREQUENCY/WAVE NUMBER AXIS **************

    fn = 1/(2*dt)
    lll=len(hepya)

    if smo == 1:
        fff = fn * np.arange(0,lll)/(ppp/2)
    elif smo == 999:
        fff = fn * np.arange(smo1a,lll+smo1a)/(ppp/2)
    else:
        fff = fn * np.arange(smo1,lll+smo1)/(ppp/2)

        
#    return winsize
    return(hepya,fff,conflim);  

# EVALUATION OBS X SAT

In [3]:
# evaluation function

def regression_results(x_true,y_true, y_pred):

    # Regression metrics
    explained_variance=explained_variance_score(y_true, y_pred)
    mae=mean_absolute_error(y_true, y_pred) 
    rmse=np.sqrt(mean_squared_error(y_true, y_pred)) 
    medae=median_absolute_error(y_true, y_pred)
    r2=r2_score(y_true, y_pred)    
    adjusted_r_squared = 1 - (1-r2)*(len(y_true)-1)/(len(y_true)-len(x_true)-1)
    print('adj r_squared', round(adjusted_r_squared,2)) 
    print('Root Mean Squared Error: ' ,round(rmse,3))
    print('R² score: ' ,round(r2,2))
    print('Average model error:', round(mae,3),'mg/L')
    return round(mae,2),round(rmse,2),round(r2,2),round(adjusted_r_squared,2)

In [28]:
# HELGOLAND ANOMALIES TIME SERIES

hplc_anom=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/helgoland_insitu_hplc_anomalies.csv')
hplc_anom['Date'] = pd.to_datetime(hplc_anom['Date'])
sat_anom=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/helgoland_globcolour_sat_anomalies.csv')
sat_anom['Date'] = pd.to_datetime(sat_anom['Date'])

In [30]:
res1 = mk.yue_wang_modification_test(hplc_anom.anomaly)
print('res1=',res1)

res2 = mk.yue_wang_modification_test(sat_anom.anomaly)
print('res2=',res2)

res1= Modified_Mann_Kendall_Test_Yue_Wang_Approach(trend='decreasing', h=True, p=0.0004332390026708932, z=-3.518961849855443, Tau=-0.12889268735644463, s=-2413.0, var_s=469813.4976907633, slope=-0.002237294456859644, intercept=0.06846867836173862)
res2= Modified_Mann_Kendall_Test_Yue_Wang_Approach(trend='decreasing', h=True, p=0.0012714031412237592, z=-3.222357672308449, Tau=-0.1128679023556434, s=-2113.0, var_s=429576.32602984953, slope=-0.001953047722278432, intercept=0.0334440044636482)


In [38]:
plt.figure(figsize=(14,6),constrained_layout=True)

h1,=plt.plot(hplc_anom.Date,hplc_anom.anomaly,'k')
plt.axhline(0, color='k')
# axs[0].set_ylabel('Chl-a anomalies (mg/m³)',y=0.1)
plt.ylabel(r'Chl-a anomalies (mg m$^{-3}$)')
plt.xlim([hplc_anom.Date.head(1),hplc_anom.Date.tail(1)])
#plt.gca().xaxis.set_major_formatter(dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_formatter(dates.DateFormatter('%Y-%m-%d'))
plt.grid(ls='--')
#axs[0].text('2018-01-01', 28, 'in situ Chl-a anomalies')
# axs[0].legend([h1,h2,h3],['positive','negative','In situ'])
m, b = np.polyfit((hplc_anom.index),(hplc_anom.anomaly), 1)
# axs[0].plot((hplc_anom.Date), (m*(hplc_anom.anomaly)+b), color='black',ls='--',lw='2',label='Linear regression')
ry=np.polyval([m,b],np.arange(1,len(hplc_anom.anomaly)+1))
res=((ry[-1]-ry[0])/22)
plt.text('2004-06-01',9, 'Trend in situ = '+str(np.round(res,3)),weight='bold')
plt.text('2003-06-01',11.5, 'a)',weight='bold')
h2,=plt.plot(sat_anom.Date,sat_anom.anomaly,'k--')
plt.legend([h1,h2],['In situ','Remote sensing'])
m, b = np.polyfit((sat_anom.index),(sat_anom.anomaly), 1)
# axs[1].plot((sat_anom.Date), (m*(sat_anom.anomaly)+b), color='black',ls='--',lw='2',label='Linear regression')
ry=np.polyval([m,b],np.arange(1,len(sat_anom.anomaly)+1))
res=((ry[-1]-ry[0])/22)
plt.text('2004-06-01',8, 'Trend remote sensing = '+str(np.round(res,3)),weight='bold')

Text(2004-06-01, 8, 'Trend remote sensing = -0.024')

In [53]:
plt.figure(figsize=(8,6),constrained_layout=True)
mae,rmse,r2,adjusted_r_squared=regression_results(hplc_anom.index,hplc_anom.anomaly,sat_anom.anomaly)
plt.plot((hplc_anom.anomaly),(sat_anom.anomaly),'.k',ms=8,label='Chl-a anomalies \n(mg m$^{-3}$)')
plt.ylim([-3,12])
plt.xlim([-3,12])
plt.ylabel('Remote Sensing')
plt.xlabel('In situ')
#axs[0].text('2013-10-01', 18, r'R$\mathregular{^2}$=0.78')
#axs[0].text('2013-10-01', 14, r'RMSE=1.113 µg L$\mathregular{^-}$$\mathregular{^1}$')
#axs[0].text(1, 22, r'a)')
plt.grid(ls='--')
#obtain m (slope) and b(intercept) of linear regression line
#m, b = np.polyfit(np.log(hplc_anom.anomaly),np.log(sat_anom.anomaly), 1)
m, b = np.polyfit((hplc_anom.anomaly),(sat_anom.anomaly), 1)
#use red as color for regression line
plt.plot((hplc_anom.anomaly), (m*(hplc_anom.anomaly)+b), color='black',label='Linear regression')
plt.axline((-3, -3), slope=1,color='black',ls='--',label='1:1')
plt.text(-2.5, 11, r'y = '+ str(round(m,2))+r'x + '+str(round(b,2)))
plt.text(-2.5, 10, r'r = '+ str(round(stats.pearsonr((hplc_anom.anomaly),(sat_anom.anomaly))[0],2)))
plt.text(-2.5, 9, r'RMSE = '+ str(round(rmse,2))+' mg m$^{-3}$')
# plt.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left",ncol=2)
plt.legend(bbox_to_anchor=(-0.04, 1, 0, 0), loc="lower left",ncol=2)
plt.gca().set_aspect('equal', 'box')
plt.xticks(np.arange(-3, 15,2));
plt.text(-5,16, 'b)',weight='bold')

adj r_squared 176.59
Root Mean Squared Error:  1.077
R² score:  0.09
Average model error: 0.695 mg/L


Text(-5, 16, 'b)')

In [31]:
# BOXPLOT MANUSCRIPT

bp3=hplc_anom[['Date','anomaly']].copy()
bp3['source']='In situ'
bp3.columns=['Date','anomaly','source']

bp4=sat_anom[['Date','anomaly']].copy()
bp4['source']='Remote sensing'
bp4.columns=['Date','anomaly','source']


bp5=pd.concat([bp3,bp4])

In [32]:
bp_count=hplc_anom.Chl_a_Hplc.groupby(hplc_anom.Date.dt.month).agg(
    ["count"]).rename_axis(
    ['month']).copy()

In [33]:
months_bp=['J','F','M','A','M','J','J','A','S','O','N','D']

In [34]:
# BOXPLOT

plt.figure(figsize=(12,6),constrained_layout=True)
ax = sns.boxplot(x=bp5.Date.dt.month, y="anomaly",hue="source", data=bp5,
                 showmeans=True, palette='colorblind',#showfliers = False,
                 meanprops={'marker': 'o','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'},
#                flierprops={'marker': 'P','markeredgecolor': 'black','markerfacecolor':'black','markersize':'5'}
                )  #
ax.legend_.set_title(None)
ax.legend(loc='upper left')
ax.set_ylabel(r'Chl-a anomalies (mg m$^{-3}$)')
ax.set_xticks(np.unique(bp5.Date.dt.month.values)-1,months_bp)

#RUN PLOT   
for item in ax.get_xticklabels():
    item.set_rotation(90)   
plt.xlabel('Month')
plt.ylim([-4,11])
plt.grid(ls='--')    
# nobs = hplc_month.values
nobs = bp_count['count'].values
nobs = [str(x) for x in nobs.tolist()]
nobs = [ii for ii in nobs]


 
# Add it to the plot
pos = range(len(nobs))
pos=[-5.5]*len(nobs)

gg=list(range(len(nobs)))
for tick in gg:
    ax.text(tick,
            pos[tick]+2,
            'N='+nobs[tick],
            horizontalalignment='center',
            size='14',
            color='k',
            weight='semibold')
plt.text(-1,11,'a)',weight='bold')

Text(-1, 11, 'a)')

In [35]:
# Distribution

plt.figure(figsize=(7,6),constrained_layout=True)
aa1=sns.histplot(data=np.clip(hplc_anom.anomaly, -2, 10+1),binwidth=1,color=[(0.00392156862745098, 0.45098039215686275, 0.6980392156862745)],
                 label='In situ') 
plt.xticks(np.arange(-3, 10+1))
aa1.set_xticklabels(np.arange(-3, 10).tolist()+[f'{10}+'])

aa2=sns.histplot(data=np.clip(sat_anom.anomaly, -2, 10+1),binwidth=1,color=[(0.8705882352941177, 0.5607843137254902, 0.0196078431372549)],
                 alpha=0.5,label='Remote sensing') 
plt.xticks(np.arange(-3, 10+1))
aa2.set_xticklabels(np.arange(-3, 10).tolist()+[f'{10}+'])
plt.legend(loc='upper right')
plt.xlabel('Chl-a anomalies (mg m$^{-3}$)')
plt.grid(ls='--')
plt.xlim([-2,11])
plt.text(-3,110,'b)',weight='bold')

Text(-3, 110, 'b)')

# DATASETS

In [20]:
ds_bat= xr.open_dataarray('/mnt/g/PhD/chapter_1/globcolour_daily/ds_bat_regrid.nc')
ds_bat_all= xr.open_dataarray('/mnt/g/PhD/chapter_1/data/ds_bat_regrid_mld_v2.nc')
xx,yy = np.meshgrid(ds_bat.lon, ds_bat.lat)

In [21]:
# CHL-A HIGHER RESOLUTION

ds_month = xr.open_dataarray('/mnt/g/PhD/chapter_1/globcolour_daily/ds_month.nc')

In [22]:
# CHL-A COMMON RESOLUTION

ds_chl=xr.open_dataarray('/mnt/g/PhD/chapter_1/data/ds_chl_regrid_v2.nc')

In [23]:
ds_chl=ds_chl.where(ds_bat_all<-5)

In [24]:
ds_chl=ds_chl.sel(lat=slice(52.95,56.05))

In [25]:
ds_chl=ds_chl.interpolate_na(dim="time", method="linear",fill_value="extrapolate")

In [26]:
# MLD DATASET

ds_mld = xr.open_dataarray('/mnt/g/PhD/chapter_1/data/ds_mld_month_v2.nc')

In [27]:
ds_mld['time']=ds_mld.time.dt.date
ds_mld['time']=pd.to_datetime(ds_mld['time'])

In [28]:
ds_mld=ds_mld.where(ds_bat_all<-5)

In [29]:
ds_mld=ds_mld.sel(lat=slice(52.95,56.05))

In [30]:
# SST DATASET

ds_sst = xr.open_dataarray('/mnt/g/PhD/chapter_1/data/ds_sst_regrid_v2.nc')

In [32]:
ds_sst=ds_sst.where(ds_bat_all<-5)

In [33]:
ds_sst=ds_sst.sel(lat=slice(52.95,56.05))

In [ ]:
# BATHYMETRY

In [34]:
ds_bat_all=ds_bat_all.sel(lat=slice(52.95,56.05))

In [ ]:
# COAST AND OCEAN MASK

In [35]:
ds_bat_coast=ds_bat_all.where((ds_bat_all>=-30) & (ds_bat_all<=-5)).copy()
ds_bat_coast_1=ds_bat.where((ds_bat>=-30) & (ds_bat<=-5)).copy()

In [36]:
land_flag = xr.where((((ds_bat_coast.lat>=53.7) & (ds_bat_coast.lon<=5)) ), True, False)
ds_bat_coast=ds_bat_coast.where(land_flag==False)
ds_bat_coast=ds_bat_coast.where(ds_bat_all.lon>=3.12)

land_flag_1 = xr.where((((ds_bat_coast_1.lat>=53.7) & (ds_bat_coast_1.lon<=5)) ), True, False)
ds_bat_coast_1=ds_bat_coast_1.where(land_flag_1==False)
ds_bat_coast_1=ds_bat_coast_1.where(ds_bat.lon>=3.12)

In [37]:
ds_bat_ocean=ds_bat_all.where(ds_bat_coast.isnull())

# BATHYMETRY CHL A RESOLUTION

ds_bat_ocean_1=ds_bat.where(ds_bat_coast_1.isnull())

# APPLY COAST OCEAN MASK

In [38]:
# CHL HIGH RESOLUTION
chl_clim_coast=ds_month.where(~ds_bat_coast_1.isnull()).mean(dim=['lon','lat'])
chl_clim_coast=chl_clim_coast.groupby('time.month').mean(dim='time')

chl_clim_ocean=ds_month.where(~ds_bat_ocean_1.isnull()).mean(dim=['lon','lat'])
chl_clim_ocean=chl_clim_ocean.groupby('time.month').mean(dim='time')

In [39]:
chl_clim_coast=ds_chl.where(~ds_bat_coast.isnull()).mean(dim=['lon','lat'])
chl_clim_coast=chl_clim_coast.groupby('time.month').mean(dim='time')

chl_clim_ocean=ds_chl.where(~ds_bat_ocean.isnull()).mean(dim=['lon','lat'])
chl_clim_ocean=chl_clim_ocean.groupby('time.month').mean(dim='time')

In [40]:
mld_clim_coast=ds_mld.where(~ds_bat_coast.isnull()).mean(dim=['lon','lat'])
mld_clim_coast=mld_clim_coast.groupby('time.month').mean(dim='time')

mld_clim_ocean=ds_mld.where(~ds_bat_ocean.isnull()).mean(dim=['lon','lat'])
mld_clim_ocean=mld_clim_ocean.groupby('time.month').mean(dim='time')


In [41]:
sst_clim_coast=ds_sst.where(~ds_bat_coast.isnull()).mean(dim=['lon','lat'])
sst_clim_coast=sst_clim_coast.groupby('time.month').mean(dim='time')

sst_clim_ocean=ds_sst.where(~ds_bat_ocean.isnull()).mean(dim=['lon','lat'])
sst_clim_ocean=sst_clim_ocean.groupby('time.month').mean(dim='time')

In [42]:
X3, Y3 = np.meshgrid(ds_bat_all.lon,ds_bat_all.lat)

XX3, YY3 = np.meshgrid(ds_bat.lon,ds_bat.lat)

In [ ]:
fig, axs = plt.subplots(1,2,subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},figsize=(13,5),sharey=True,constrained_layout=True)
axs=axs.flatten()
img=axs[0].pcolormesh(X3,Y3,ds_bat_ocean.where(ds_bat_ocean<-5),vmin=-50,vmax=-30,cmap='viridis',transform=ccrs.PlateCarree())
CS=axs[0].contour(X3,Y3,ds_bat_all,levels=[-30], colors='gray',linestyles=['solid'],transform=ccrs.PlateCarree())
axs[0].clabel(CS, inline=True,fontsize=18, fmt='%1.0f',colors='k')
axs[0].set_xlabel('Longitude (°)')
axs[0].set_ylabel('Latitude (°)')
axs[0].add_feature(cfeature.LAND, facecolor='grey')
axs[0].coastlines()
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_yticklabels([]) # -90, -60, -30
axs[0].set_xticklabels([3, 6, 9])
cbar=plt.colorbar(img,shrink=0.86,pad=0.01,ax=axs[0])
                  
img1=axs[1].pcolormesh(X3,Y3,ds_bat_coast.where(ds_bat_coast<-5),vmin=-30,vmax=-5,cmap='viridis',transform=ccrs.PlateCarree())
CS1=axs[1].contour(X3,Y3,ds_bat_all,levels=[-30], colors='gray',linestyles=['solid'],transform=ccrs.PlateCarree())
axs[1].clabel(CS1, inline=True,fontsize=18, fmt='%1.0f',colors='k')
axs[1].set_xlabel('Longitude (°)')
#axs[1].set_ylabel('Latitude (°)')
axs[1].add_feature(cfeature.LAND, facecolor='grey')
axs[1].coastlines()
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_yticklabels([]) # -90, -60, -30
axs[1].set_xticklabels([3, 6, 9])
cbar=plt.colorbar(img1,shrink=0.86,pad=0.01,ax=axs[1],label='Bathymetry (m)')
axs[0].set_title('Offshore')
axs[1].set_title('Coast')

In [43]:
# CHL HIGH RESOLUTION
chl_clim_coast=ds_month.where(~ds_bat_coast_1.isnull()).mean(dim=['lon','lat'])
chl_clim_coast=chl_clim_coast.groupby('time.month').mean(dim='time')

chl_clim_ocean=ds_month.where(~ds_bat_ocean_1.isnull()).mean(dim=['lon','lat'])
chl_clim_ocean=chl_clim_ocean.groupby('time.month').mean(dim='time')

In [58]:
# HIGHER RESOLUTION #####
# Monthly climatology and anomaly\n",
chl_clim = ds_month.groupby('time.month').mean(dim='time')
#chl_mean = ds_month.mean(dim='time')
#chl_std = ds_month.std(dim='time')
chl_anomalies = ds_month.groupby('time.month') - chl_clim
#chl_anom_mean = chl_anomalies.mean(dim='time')
#chl_anom_std = chl_anomalies.std(dim='time')

In [45]:
# Monthly climatology and anomaly"
clim_chl = ds_month.groupby('time.month').mean(dim='time')
chl_mean = ds_month.mean(dim='time')
chl_std = ds_month.std(dim='time')
anomalies = ds_month.groupby('time.month') - clim_chl
anom_mean = anomalies.mean(dim='time')
anom_std = anomalies.std(dim='time')

/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [13]:
# Linear Trend

result_trend=[]
p_value=[]

for lat in range(0,len(anomalies.lat)):
    for lon in range(0,len(anomalies.lon)):
        try:
            reg = np.polyfit(np.arange(1,len(anomalies)+1),anomalies[:,lat,lon],1)
            ry=np.polyval(reg,np.arange(1,len(anomalies)+1))
            res=((ry[-1]-ry[0])/(22))
            slope, intercept, r_value, pp, std_err = sp.stats.linregress(np.arange(1,len(anomalies)+1),anomalies[:,lat,lon])
        except ZeroDivisionError:
                res = np.nan
                pp = np.nan
        #print(lat,lon)
        result_trend.append((res, float(anomalies.lat[lat]), float(anomalies.lon[lon])))
        p_value.append((pp, float(anomalies.lat[lat]), float(anomalies.lon[lon])))        

result_trend=pd.DataFrame(result_trend, columns=('trend', 'lat', 'lon'))
p_value=pd.DataFrame(p_value, columns=('p_value', 'lat', 'lon'))

In [14]:
0.77/2.2

0.35

In [15]:
pv_result_trend= pd.pivot_table(result_trend, index=result_trend.lat, columns=result_trend.lon,values='trend')

In [16]:
X1, Y1 = np.meshgrid(pv_result_trend.columns,pv_result_trend.index)

In [17]:
# Define the figure and each axis for the 3 rows and 4 columns
fig, axs = plt.subplots(nrows=2,ncols=2,
                        subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},
                        figsize=(13,9),constrained_layout=True)

# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#fig.subplots_adjust(bottom=0.1, top=0.98, left=0.05, right=0.99,
#                    wspace=0.0, hspace=0.05)

img=chl_mean.lon,chl_mean.lat,(chl_mean).plot.pcolormesh(ax=axs[0],cmap='viridis',vmin=0,vmax=5,
                                    add_colorbar=True,
                                    # cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(mg m$^\N{MINUS SIGN}$$^3$)'},
                                    cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(mg m$^{-3}$)'},                                                         
                                                                 extend='neither',transform=ccrs.PlateCarree())#
#m = mpl_toolkits.basemap(projection='lcc', width=12000000, height=9000000,
#resolution='c', lat_1=47.536042,lat_2=23.878382,lat_0=56.362771,lon_0=30.517389)
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_xlabel(None)
axs[0].set_ylabel('Latitude (°)')
#ax.plot(lon_e,lat_e,'k')
#ax.set_ylim([53,56])
#ax.set_xlim([2.5,9.25])
CS10=axs[0].contour(xx,yy,ds_bat,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30=axs[0].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(2)], colors='r',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(1)], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
CS1=axs[0].contour(xx,yy,(chl_mean),levels=[2], colors='r',transform=ccrs.PlateCarree())
h3,_ = CS1.legend_elements()
CS2=axs[0].contour(xx,yy,(chl_mean),levels=[1], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
h4,_ = CS2.legend_elements()
axs[0].add_feature(cfeature.LAND, facecolor='grey')
axs[0].coastlines()
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[0].set_xticklabels([])    
#plt.legend([h4[0],h3[0]],['1 mg m$^\N{MINUS SIGN}$$^3$','2 mg m$^\N{MINUS SIGN}$$^3$'])
axs[0].text(2.4,56.05,'a)',weight='bold',transform=ccrs.PlateCarree())

img2=chl_std.lon,chl_std.lat,(chl_std).plot.contourf(ax=axs[1],levels=[0,1,2,3,4,5],cmap='viridis',vmin=0,vmax=5,
                                    add_colorbar=True,
                                    # cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(mg m$^\N{MINUS SIGN}$$^3$)'},
                                    cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(mg m$^{-3}$)'},                                                     
                                                                 extend='neither',transform=ccrs.PlateCarree())#
#m = mpl_toolkits.basemap(projection='lcc', width=12000000, height=9000000,
#resolution='c', lat_1=47.536042,lat_2=23.878382,lat_0=56.362771,lon_0=30.517389)
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_xlabel('Longitude (°)')
axs[1].set_ylabel(None)
#ax.plot(lon_e,lat_e,'k')
#ax.set_ylim([53,56])
#ax.set_xlim([2.5,9.25])
CS10a=axs[1].contour(xx,yy,ds_bat,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30a=axs[1].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(2)], colors='r',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(1)], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
CSa=axs[1].contour(xx,yy,(chl_std),levels=[2], colors='r',transform=ccrs.PlateCarree())
h1,_ = CSa.legend_elements()
CSb=axs[1].contour(xx,yy,(chl_std),levels=[1], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
h2,_ = CSb.legend_elements()
axs[1].add_feature(cfeature.LAND, facecolor='grey')
axs[1].coastlines()
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_yticklabels([]) # -90, -60, -30
axs[1].set_xticklabels([3, 6, 9]) 
axs[1].set_ylabel(None)
axs[1].set_xlabel(None)
# axs[1].legend([h2[0],h1[0]],['1 mg m$^\N{MINUS SIGN}$$^3$','2 mg m$^\N{MINUS SIGN}$$^3$'],loc='upper left')
# axs[1].legend([h2[0],h1[0]],['1 mg/m³','2 mg/m³'],loc='upper left')
axs[1].legend([h2[0],h1[0]],['1 mg m$^{-3}$','2 mg m$^{-3}$'],loc='upper left')
axs[1].set_xticklabels([])    
axs[1].text(2.4,56.05,'b)',weight='bold',transform=ccrs.PlateCarree())

img3=chl_std.lon,chl_std.lat,((chl_std/chl_mean)*100).plot.contourf(ax=axs[2],cmap='viridis',#vmin=0,vmax=5,
                                    add_colorbar=True,levels=list(np.arange(0,110,10)),
                                    cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(%)'},
                                                                 extend='neither',transform=ccrs.PlateCarree())#
CS10a=axs[2].contour(xx,yy,ds_bat,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30a=axs[2].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
axs[2].add_feature(cfeature.LAND, facecolor='grey')
axs[2].coastlines()
axs[2].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[2].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[2].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[2].set_xticklabels([3, 6, 9]) 
axs[2].set_ylabel('Latitude (°)')
axs[2].set_xlabel('Longitude (°)')
axs[2].text(2.4,56.05,'c)',weight='bold',transform=ccrs.PlateCarree())

divnorm = mclr.TwoSlopeNorm(vmin=-0.03, vcenter=0, vmax=0.03)
img4=axs[3].pcolormesh(X1,Y1,pv_result_trend.values,shading='auto', norm=divnorm,cmap='seismic',transform=ccrs.PlateCarree())
cbar=plt.colorbar(img4,orientation='vertical',pad=0.01,shrink=0.90)
# cbar.set_label('(mg/m³)/year')
cbar.set_label('(mg m$^{-3}$) year$^{-1}$')
CS10=axs[3].contour(xx,yy,ds_bat,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30=axs[3].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
#plt.plot(result.lon[result.trend==0.25],result.lat[result.trend==0.25],'.k',transform=ccrs.PlateCarree(),markersize=0.5)
#plt.plot(result.lon[result.trend==-0.5],result.lat[result.trend==-0.5],'.k',transform=ccrs.PlateCarree(),markersize=0.5)
axs[3].plot(p_value.lon[p_value.p_value<=0.05],p_value.lat[p_value.p_value<=0.05],'.k',transform=ccrs.PlateCarree(),markersize=0.2)
axs[3].set_xlabel('Longitude (°)')
axs[3].set_ylabel(None)
axs[3].add_feature(cfeature.LAND, facecolor='grey')
axs[3].coastlines()
axs[3].set_yticks([ 53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[3].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[3].set_yticklabels([]) # -90, -60, -30
axs[3].set_xticklabels([3, 6, 9]) # 3, 6, 9
axs[3].text(2.4,56.05,'d)',weight='bold',transform=ccrs.PlateCarree())


Text(2.4, 56.05, 'd)')

In [17]:
months=['J','F','M','A','M','J','J','A','S','O','N','D']
levels=[-30]

In [ ]:
# Define the figure and each axis for the 3 rows and 4 columns
fig, axs = plt.subplots(nrows=3,ncols=4,
                        subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},
                        figsize=(12.5,7))

# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

fig.subplots_adjust(bottom=0.15, top=0.95, left=0.1, right=0.9,
                    wspace=0.0, hspace=0.05)
nn=0

for i in range(0,12):

    img=clim_chl.sel(month=clim_chl.month.values[nn]).plot.pcolormesh(
        ax=axs[i], vmin=0, vmax=5, cmap='jet',add_colorbar=False,add_labels=False, extend='both',transform=ccrs.PlateCarree())
#                cbar_kwargs={
#        "orientation": "vertical",
#        "label": "Chl-a (mg m$\\mathregular{^-}$$\\mathregular{^3}$)\",
#        "pad": 0.01,
#    })

    CS=axs[i].contour(xx,yy,ds_bat,levels=levels, colors='k',transform=ccrs.PlateCarree())

    axs[i].clabel(CS, inline=True,fontsize=20, fmt='%1.0f')

    axs[i].set_ylabel(None)
    axs[i].set_xlabel(None)
    axs[i].text(0.91, 0.86, str(months[nn]),transform=axs[i].transAxes,ha='left')
#        axes[i, j].set_yticks([54,56],transform=ccrs.PlateCarree())
    axs[i].add_feature(cfeature.LAND, facecolor='grey')
    axs[i].coastlines()
    nn=nn+1
# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.9, 0.15, 0.02, 0.8])    
cbar=fig.colorbar(img, cax=cbar_ax,orientation='vertical')

axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[4].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[4].set_yticklabels([53, 54,55,'']) # -90, -60, -30
axs[8].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[8].set_yticklabels([53, 54,55,'']) # -90, -60, -30

axs[8].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[8].set_xticklabels([3, 6, 9])
axs[9].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[9].set_xticklabels([3, 6, 9])
axs[10].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[10].set_xticklabels([3, 6, 9])
axs[11].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[11].set_xticklabels([3, 6, 9])

# cbar.set_label('Chl-a (mg m$\mathregular{^-}$$\mathregular{^3}$)')
cbar.set_label('Chl-a (mg m$^{-3}$)')
fig.text(0.05, 0.47, 'Latitude (°)', ha='left',rotation='vertical')
fig.text(0.46, 0.05, 'Longitude (°)', ha='left',rotation='horizontal')

In [18]:
# Mann-kendall trend significance test

result=[]

for lat in range(0,len(chl_anomalies.lat)):
    for lon in range(0,len(chl_anomalies.lon)):
        try:
            # res = mk.original_test(anomalies[:,lat,lon])
            # res = mk.hamed_rao_modification_test(chl_anomalies[:,lat,lon])
            res = mk.yue_wang_modification_test(chl_anomalies[:,lat,lon])
            
            
            if res[0]=='decreasing':
                trend=-0.5
            
            elif res[0]=='no trend':
                if res[5]<0:
                    trend=-0.25
                else:
                    trend=0 
                            
            else:
                trend=0.25
        except ZeroDivisionError:
                trend = np.nan
        
#        print(lat,lon)
        result.append((trend, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon])))

result=pd.DataFrame(result, columns=('trend', 'lat', 'lon'))

/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/pymannkendall/pymannkendall.py:60: RuntimeWarning: invalid value encountered in divide
  return acov[:nlags+1]/acov[0]


In [19]:
pv_result= pd.pivot_table(result, index=result.lat, columns=result.lon,values='trend')

In [20]:
import matplotlib.colors as clb

In [21]:
cmap = mclr.ListedColormap(['b', 'purple','orange','r'])
# X, Y = np.meshgrid(pv_result_yue_wang.columns,pv_result_yue_wang.index)
X, Y = np.meshgrid(pv_result.columns,pv_result.index)
# X1, Y1 = np.meshgrid(pv_result_hamed_rao.columns,pv_result_hamed_rao.index)
X=X.astype('float64')
Y=Y.astype('float64')
# X1=X1.astype('float64')
# Y1=Y1.astype('float64')
bounds = [-0.5,-0.25,0,0.25]
ticks = [-0.4125,-0.225,-0.025,0.15]

In [22]:
fig=plt.figure(figsize=(14,8),constrained_layout=True)
ax=plt.axes(projection=ccrs.Mercator(central_longitude=7.1))
img=plt.pcolormesh(X,Y,pv_result.values,shading='auto', cmap=cmap,transform=ccrs.PlateCarree())
# img=plt.pcolormesh(X1,Y1,pv_result_hamed_rao.values,shading='auto', cmap=cmap,transform=ccrs.PlateCarree())
cbar=plt.colorbar(orientation='horizontal', ticks = ticks,shrink=0.59,pad=0.01)
plt.clim([-0.5,0.25])
cbar.set_ticklabels(['decreasing','dec. no sig.','inc. no sig.','increasing'])
# CS=ax.contour(xx,yy,ds_bat,levels=[-30], colors='black',linestyles=['dashed'],transform=ccrs.PlateCarree())
# CS1=ax.contour(xx,yy,ds_bat,levels=[-10], colors='grey',linestyles=['dashed'],transform=ccrs.PlateCarree())
plt.xlabel('Longitude (°)')
plt.ylabel('Latitude (°)')
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.coastlines()
ax.set_yticks([ 53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
ax.set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
ax.set_yticklabels([53, 54,55,56]) # -90, -60, -30
ax.set_xticklabels([3, 6, 9]) # 3, 6, 9
# plt.title('Yue_wang')

[Text(-456409.9122524203, 0, '3'),
 Text(-122451.43987259835, 0, '6'),
 Text(211507.03250722372, 0, '9')]

In [23]:
print(pv_result[pv_result==-0.5].count().sum()) # decreasing sig
print(pv_result[pv_result==-0.25].count().sum()) # decreasing not sig
print(pv_result[pv_result==0.].count().sum()) # increasing not sig
print(pv_result[pv_result==0.25].count().sum()) # increasing sig

77574
36495
19649
8387


In [24]:
(pv_result[pv_result==-0.5].count().sum())+(pv_result[pv_result==-0.25].count().sum())+(pv_result[pv_result==0.].count().sum())+(pv_result[pv_result==0.25].count().sum())

142105

In [29]:
8387/pv_result.count().sum()

0.059019738925442454

In [25]:
pv_result.count().sum()

142105

# SST trends

In [ ]:
ds_bat_all= xr.open_dataarray('/mnt/g/PhD/chapter_1/data/ds_bat_regrid_mld_v2.nc')

In [ ]:
xx,yy = np.meshgrid(ds_bat_all.lon, ds_bat_all.lat)

In [ ]:
ds_sst = xr.open_dataarray('/mnt/g/PhD/chapter_1/data/ds_sst_regrid_v2.nc')

In [ ]:
ds_sst=ds_sst.where(ds_bat_all<-5)

In [ ]:
ds_sst=ds_sst.sel(lat=slice(52.95,56.05))

In [ ]:
ds_bat_all=ds_bat_all.sel(lat=slice(52.95,56.05))

In [ ]:
sst_clim = ds_sst.groupby('time.month').mean(dim='time')
sst_mean = ds_sst.mean(dim='time')
sst_std = ds_sst.std(dim='time')
sst_anomalies = ds_sst.groupby('time.month') - sst_clim
sst_anom_mean = sst_anomalies.mean(dim='time')
#sst_anom_std = sst_anomalies.std(dim='time')

In [ ]:
# Linear Trend

result_trend=[]
p_value=[]

for lat in range(0,len(sst_anomalies.lat)):
    for lon in range(0,len(sst_anomalies.lon)):
        try:
            reg = np.polyfit(np.arange(1,len(sst_anomalies)+1),sst_anomalies[:,lat,lon],1)
            ry=np.polyval(reg,np.arange(1,len(sst_anomalies)+1))
            res=((ry[-1]-ry[0])/(22))
            slope, intercept, r_value, pp, std_err = sp.stats.linregress(np.arange(1,len(sst_anomalies)+1),sst_anomalies[:,lat,lon])
        except ZeroDivisionError:
                res = np.nan
                pp = np.nan
        #print(lat,lon)
        result_trend.append((res, float(sst_anomalies.lat[lat]), float(sst_anomalies.lon[lon])))
        p_value.append((pp, float(sst_anomalies.lat[lat]), float(sst_anomalies.lon[lon])))        

result_trend=pd.DataFrame(result_trend, columns=('trend', 'lat', 'lon'))
p_value=pd.DataFrame(p_value, columns=('p_value', 'lat', 'lon'))

In [ ]:
pv_result_trend= pd.pivot_table(result_trend, index=result_trend.lat, columns=result_trend.lon,values='trend')

In [ ]:
X1, Y1 = np.meshgrid(pv_result_trend.columns,pv_result_trend.index)

In [ ]:
# Define the figure and each axis for the 3 rows and 4 columns
fig, axs = plt.subplots(nrows=2,ncols=2,
                        subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},
                        figsize=(13,9),constrained_layout=True)

# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

#fig.subplots_adjust(bottom=0.1, top=0.98, left=0.05, right=0.99,
#                    wspace=0.0, hspace=0.05)

img=sst_mean.lon,sst_mean.lat,(sst_mean).plot.pcolormesh(ax=axs[0],cmap='viridis',
                                    add_colorbar=True,
                                    # cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(mg m$^\N{MINUS SIGN}$$^3$)'},
                                    cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(°C)'},                                                         
                                                                 extend='neither',transform=ccrs.PlateCarree())#
#m = mpl_toolkits.basemap(projection='lcc', width=12000000, height=9000000,
#resolution='c', lat_1=47.536042,lat_2=23.878382,lat_0=56.362771,lon_0=30.517389)
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_xlabel(None)
axs[0].set_ylabel('Latitude (°)')
#ax.plot(lon_e,lat_e,'k')
#ax.set_ylim([53,56])
#ax.set_xlim([2.5,9.25])
CS10=axs[0].contour(xx,yy,ds_bat_all,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30=axs[0].contour(xx,yy,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(2)], colors='r',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(1)], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
CS1=axs[0].contour(xx,yy,(sst_mean),levels=[15], colors='r',transform=ccrs.PlateCarree())
h3,_ = CS1.legend_elements()
CS2=axs[0].contour(xx,yy,(sst_mean),levels=[10], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
h4,_ = CS2.legend_elements()
axs[0].add_feature(cfeature.LAND, facecolor='grey')
axs[0].coastlines()
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[0].set_xticklabels([])    
#plt.legend([h4[0],h3[0]],['1 mg m$^\N{MINUS SIGN}$$^3$','2 mg m$^\N{MINUS SIGN}$$^3$'])
axs[0].text(2.4,56.05,'a)',weight='bold',transform=ccrs.PlateCarree())

img2=sst_std.lon,sst_std.lat,(sst_std).plot.contourf(ax=axs[1],levels=list(np.arange(3,7,0.1)),cmap='viridis',
                                    add_colorbar=True,
                                    # cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(mg m$^\N{MINUS SIGN}$$^3$)'},
                                    cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(°C)'},                                                     
                                                                 extend='neither',transform=ccrs.PlateCarree())#
#m = mpl_toolkits.basemap(projection='lcc', width=12000000, height=9000000,
#resolution='c', lat_1=47.536042,lat_2=23.878382,lat_0=56.362771,lon_0=30.517389)
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_xlabel('Longitude (°)')
axs[1].set_ylabel(None)
#ax.plot(lon_e,lat_e,'k')
#ax.set_ylim([53,56])
#ax.set_xlim([2.5,9.25])
CS10a=axs[1].contour(xx,yy,ds_bat_all,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30a=axs[1].contour(xx,yy,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(2)], colors='r',transform=ccrs.PlateCarree())
#CS=ax.contour(xx,yy,np.log10(chl_mean),levels=[np.log10(1)], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
CSa=axs[1].contour(xx,yy,(sst_std),levels=[5], colors='r',transform=ccrs.PlateCarree())
h1,_ = CSa.legend_elements()
CSb=axs[1].contour(xx,yy,(sst_std),levels=[3], linestyles='dashed',colors='r',transform=ccrs.PlateCarree())
h2,_ = CSb.legend_elements()
axs[1].add_feature(cfeature.LAND, facecolor='grey')
axs[1].coastlines()
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_yticklabels([]) # -90, -60, -30
axs[1].set_xticklabels([3, 6, 9]) 
axs[1].set_ylabel(None)
axs[1].set_xlabel(None)
# axs[1].legend([h2[0],h1[0]],['1 mg m$^\N{MINUS SIGN}$$^3$','2 mg m$^\N{MINUS SIGN}$$^3$'],loc='upper left')
# axs[1].legend([h2[0],h1[0]],['1 mg/m³','2 mg/m³'],loc='upper left')
axs[1].legend([h2[0],h1[0]],['1 °C','2 °C'],loc='upper left')
axs[1].set_xticklabels([])    
axs[1].text(2.4,56.05,'b)',weight='bold',transform=ccrs.PlateCarree())

img3=sst_std.lon,sst_std.lat,((sst_std/sst_mean)*100).plot.contourf(ax=axs[2],cmap='viridis',#vmin=0,vmax=5,
                                    add_colorbar=True,levels=list(np.arange(0,110,10)),
                                    cbar_kwargs={'shrink': 0.90,'pad':0.01,'label':'(%)'},
                                                                 extend='neither',transform=ccrs.PlateCarree())#
CS10a=axs[2].contour(xx,yy,ds_bat_all,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30a=axs[2].contour(xx,yy,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
axs[2].add_feature(cfeature.LAND, facecolor='grey')
axs[2].coastlines()
axs[2].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[2].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[2].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[2].set_xticklabels([3, 6, 9]) 
axs[2].set_ylabel('Latitude (°)')
axs[2].set_xlabel('Longitude (°)')
axs[2].text(2.4,56.05,'c)',weight='bold',transform=ccrs.PlateCarree())

divnorm = mclr.TwoSlopeNorm(vmin=-0.001, vcenter=0, vmax=0.035)
img4=axs[3].pcolormesh(X1,Y1,pv_result_trend.values,shading='auto', norm=divnorm,cmap='seismic',transform=ccrs.PlateCarree())
cbar=plt.colorbar(img4,orientation='vertical',pad=0.01,shrink=0.90)
# cbar.set_label('(mg/m³)/year')
cbar.set_label('(mg m$^{-3}$) year$^{-1}$')
CS10=axs[3].contour(xx,yy,ds_bat_all,levels=[-10], colors='grey',linestyles='dashed',transform=ccrs.PlateCarree())
CS30=axs[3].contour(xx,yy,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
#plt.plot(result.lon[result.trend==0.25],result.lat[result.trend==0.25],'.k',transform=ccrs.PlateCarree(),markersize=0.5)
#plt.plot(result.lon[result.trend==-0.5],result.lat[result.trend==-0.5],'.k',transform=ccrs.PlateCarree(),markersize=0.5)
axs[3].plot(p_value.lon[p_value.p_value<=0.05],p_value.lat[p_value.p_value<=0.05],'.k',transform=ccrs.PlateCarree(),markersize=0.2)
axs[3].set_xlabel('Longitude (°)')
axs[3].set_ylabel(None)
axs[3].add_feature(cfeature.LAND, facecolor='grey')
axs[3].coastlines()
axs[3].set_yticks([ 53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[3].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[3].set_yticklabels([]) # -90, -60, -30
axs[3].set_xticklabels([3, 6, 9]) # 3, 6, 9
axs[3].text(2.4,56.05,'d)',weight='bold',transform=ccrs.PlateCarree())


In [ ]:
# Mann-kendall trend significance test

result=[]

for lat in range(0,len(sst_anomalies.lat)):
    for lon in range(0,len(sst_anomalies.lon)):
        try:
            # res = mk.original_test(sst_anomalies[:,lat,lon])
            # res = mk.hamed_rao_modification_test(sst_anomalies[:,lat,lon])
            res = mk.yue_wang_modification_test(sst_anomalies[:,lat,lon])
            
            
            if res[0]=='decreasing':
                trend=-0.5
            
            elif res[0]=='no trend':
                if res[5]<0:
                    trend=-0.25
                else:
                    trend=0 
                            
            else:
                trend=0.25
        except ZeroDivisionError:
                trend = np.nan
        
#        print(lat,lon)
        result.append((trend, float(sst_anomalies.lat[lat]), float(sst_anomalies.lon[lon])))

result=pd.DataFrame(result, columns=('trend', 'lat', 'lon'))

In [ ]:
pv_result= pd.pivot_table(result, index=result.lat, columns=result.lon,values='trend')

In [ ]:
import matplotlib.colors as clb

In [ ]:
cmap = mclr.ListedColormap(['b', 'purple','orange','r'])
X, Y = np.meshgrid(pv_result.columns,pv_result.index)
X=X.astype('float64')
Y=Y.astype('float64')
bounds = [-0.5,-0.25,0,0.25]
ticks = [-0.4125,-0.225,-0.025,0.15]

In [ ]:
float(pv_result.columns[0])

In [ ]:
fig=plt.figure(figsize=(14,8),constrained_layout=True)
ax=plt.axes(projection=ccrs.Mercator(central_longitude=7.1))
#ax.set_ylim([53,56])
#ax.set_xlim([2.5,9.25])
#plt.pcolormesh(pv_sylt_oisst.columns,pv_sylt_oisst.index,pv_sylt_oisst.values,shading='auto', cmap=plt.cm.jet)
img=plt.pcolormesh(X,Y,pv_result.values,shading='auto', cmap=cmap,transform=ccrs.PlateCarree())
cbar=plt.colorbar(orientation='horizontal', ticks = ticks,shrink=0.59,pad=0.01)
plt.clim([-0.5,0.25])
cbar.set_ticklabels(['decreasing','dec. no sig.','inc. no sig.','increasing'])
CS=ax.contour(xx,yy,ds_bat_all,levels=[-30], colors='black',linestyles=['dashed'],transform=ccrs.PlateCarree())
CS1=ax.contour(xx,yy,ds_bat_all,levels=[-10], colors='grey',linestyles=['dashed'],transform=ccrs.PlateCarree())
plt.xlabel('Longitude (°)')
plt.ylabel('Latitude (°)')
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.coastlines()
ax.set_yticks([ 53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
ax.set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
ax.set_yticklabels([53, 54,55,56]) # -90, -60, -30
ax.set_xticklabels([3, 6, 9]) # 3, 6, 9

In [ ]:
sst_time_mean=sst_anomalies.mean(dim=['lat','lon'])

In [ ]:
sst_time_mean=sst_time_mean.to_dataframe().reset_index(drop=False)

In [ ]:
plt.figure(constrained_layout=True,figsize=[9,4])
a1,=plt.plot(sst_time_mean.time,sst_time_mean.analysed_sst,'.-',color='grey',label='SST anomalies')
a2,=plt.plot(sst_time_mean.time,stl[:,1],'black',label='LOWESS trend')
plt.plot(sst_time_mean.time.iloc[0],stl[0,1],'.',color='black',markersize=10)
plt.plot(sst_time_mean.time.iloc[-1],stl[-1,1],'.',color='black',markersize=10)
plt.xlim([sst_time_mean.time.iloc[0],sst_time_mean.time.iloc[-1]])
plt.xlim(['1997-12-01','2021-01-31'])
plt.axhline(y=0,lw=1)
plt.ylabel('Temperature (°C)')
plt.grid(ls='--')
plt.legend(ncol=2)


# ANALYSIS

# INVESTIGATE TEMPORAL CHANGES

In [48]:
month_l=['Date','jan','feb','march','april','may','jun','jul','aug','sep','oct','nov','dec']

In [ ]:
# april_ocean=chl_anomalies[chl_anomalies.time.dt.month==4].where(ds_bat<-30).mean(dim=['lon','lat'])
# april_coast=chl_anomalies[chl_anomalies.time.dt.month==4].where(ds_bat>=-30).mean(dim=['lon','lat'])

# april_ocean=chl_anomalies[chl_anomalies.time.dt.month==4].where(ds_bat<-30).mean(dim=['lon','lat'])
# april_coast=chl_anomalies[chl_anomalies.time.dt.month==4].where(ds_bat>=-30).mean(dim=['lon','lat'])



In [56]:
years=list(np.arange(1998,2021))

In [57]:
m_idx=list(np.arange(1,13))

In [59]:
#TIME SERIES EACH MONTH - COAST OFF SHORE ANOMALIES

res_coast=pd.DataFrame(columns=month_l)
res_ocean=pd.DataFrame(columns=month_l)
std_coast=pd.DataFrame(columns=month_l)
std_ocean=pd.DataFrame(columns=month_l)

res_coast['Date']=years
res_ocean['Date']=years
std_coast['Date']=years
std_ocean['Date']=years



for mm in m_idx:
    
    oo=chl_anomalies[chl_anomalies.time.dt.month==mm].where(~ds_bat_ocean_1.isnull()).mean(dim=['lon','lat']) # OFF SHORE
    cc=chl_anomalies[chl_anomalies.time.dt.month==mm].where(~ds_bat_coast_1.isnull()).mean(dim=['lon','lat']) # COAST
    oo1=chl_anomalies[chl_anomalies.time.dt.month==mm].where(~ds_bat_ocean_1.isnull()).std(dim=['lon','lat']) # OFF SHORE
    cc1=chl_anomalies[chl_anomalies.time.dt.month==mm].where(~ds_bat_coast_1.isnull()).std(dim=['lon','lat']) # COAST
    
    #res_coast.iloc[:,mm-1]=cc
    res_coast[res_coast.columns[mm]]=cc
    #res_ocean.iloc[:,mm-1]=oo
    res_ocean[res_ocean.columns[mm]]=oo
    
    std_coast[std_coast.columns[mm]]=cc1
    std_ocean[std_ocean.columns[mm]]=oo1
    
    del(cc,oo,cc1,oo1)


In [60]:
sat_coast=chl_anomalies.where(~ds_bat_coast_1.isnull()).mean(dim=['lon','lat']).to_dataframe().reset_index(drop=False) # COAST
sat_ocean=chl_anomalies.where(~ds_bat_ocean_1.isnull()).mean(dim=['lon','lat']).to_dataframe().reset_index(drop=False)  # OFF SHORE

In [61]:
print('March coast=',mk.yue_wang_modification_test(res_coast.iloc[:,3]))
print('April coast=',mk.yue_wang_modification_test(res_coast.iloc[:,4]))
print('May coast=',mk.yue_wang_modification_test(res_coast.iloc[:,5]))

print('March ocean=',mk.yue_wang_modification_test(res_ocean.iloc[:,3]))
print('April ocean=',mk.yue_wang_modification_test(res_ocean.iloc[:,4]))
print('May ocean=',mk.yue_wang_modification_test(res_ocean.iloc[:,5]))


March coast= Modified_Mann_Kendall_Test_Yue_Wang_Approach(trend='no trend', h=False, p=0.06227921317086915, z=1.864302662294392, Tau=0.16996047430830039, s=43.0, var_s=507.5347012627887, slope=0.016901366412639618, intercept=-0.34972115606069565)
April coast= Modified_Mann_Kendall_Test_Yue_Wang_Approach(trend='no trend', h=False, p=0.26548763647486195, z=-1.1135142387912174, Tau=-0.09090909090909091, s=-23.0, var_s=390.3496658781044, slope=-0.015566051006317139, intercept=0.16547604789957404)
May coast= Modified_Mann_Kendall_Test_Yue_Wang_Approach(trend='decreasing', h=True, p=0.007447713861839622, z=-2.6761327520192277, Tau=-0.24110671936758893, s=-61.0, var_s=502.67489323468675, slope=-0.0440611583845956, intercept=0.4010356834956578)
March ocean= Modified_Mann_Kendall_Test_Yue_Wang_Approach(trend='increasing', h=True, p=0.0010022984221436104, z=3.2898807735402404, Tau=0.20948616600790515, s=53.0, var_s=249.83102441883563, slope=0.006508633494377136, intercept=-0.08904029242694378)
A

In [62]:
# NAO

nao=pd.read_csv('/mnt/g/PhD/chapter_1/data/nao_winter.csv')

In [66]:
# ANOMALIES CHLOROPHYLL-A

fig,axs = plt.subplots(2, 1, figsize=(14,8),constrained_layout=True,sharex=True)
axs=axs.flatten()
axs[0].plot(res_coast.Date,res_coast.iloc[:,3],'s-',ms='6',label='March');
axs[0].plot(res_coast.Date,res_coast.iloc[:,4],'.-',ms='12',label='April');
axs[0].plot(res_coast.Date,res_coast.iloc[:,5],'^-',ms='8',label='May');
axs[0].legend(title='Chl-a coast',ncol=3)
axs[0].grid(ls='--')
axs[0].set_xticks(years);
axs[0].set_xticklabels([]);


axs[1].plot(res_ocean.Date,res_ocean.iloc[:,3],'s-',ms='6',label='March');
axs[1].plot(res_ocean.Date,res_ocean.iloc[:,4],'.-',ms='12',label='April');
axs[1].plot(res_ocean.Date,res_ocean.iloc[:,5],'^-',ms='8',label='May');
axs[1].legend(title='Chl-a offshore',loc='upper right',ncol=3,bbox_to_anchor=(1.01,1.1))
axs[1].grid(ls='--')
axs[1].set_xticks(years);
axs[1].set_ylabel('Chl-a (mg/m³)');
axs[1].set_xticks(res_ocean.Date,rotation=45);
axs[1].set_xticklabels(years,rotation=45);

In [67]:
# ANOMALIES CHLOROPHYLL-A WITH NAO

fig,axs = plt.subplots(2, 1, figsize=(14,6),constrained_layout=True,sharex=True)
axs=axs.flatten()
pp1,=axs[0].plot(res_coast.Date,res_coast.iloc[:,3],'s-',ms='6',label='March');
pp2,=axs[0].plot(res_coast.Date,res_coast.iloc[:,4],'.-',ms='12',label='April');
pp3,=axs[0].plot(res_coast.Date,res_coast.iloc[:,5],'^-',ms='8',label='May');
# axs[0].legend(title='Chl-a coast',ncol=3)
axs[0].grid(ls='--')
axs[0].set_xticks(years);
axs[0].set_xticklabels([]);
axs[0].text(0.005,0.88,'a)',weight='bold',transform=axs[0].transAxes)
axs[0].text(0.03,0.875,'Coast',transform=axs[0].transAxes)

twin1=axs[0].twinx()

pp4,=twin1.plot(nao.Date[nao.Date<=2020],nao.NAO_DJF[nao.Date<=2020],'k',lw='0.5',label='NAO winter')
# twin1.set_ylabel('NAO index')
twin1.legend([pp1,pp2,pp3,pp4],['March','April','May','NAO winter'],ncol=4,loc='upper left',bbox_to_anchor=(0,1.3))
twin1.grid(ls='dotted')

pp5,=axs[1].plot(res_ocean.Date,res_ocean.iloc[:,3],'s-',ms='6',label='March');
pp6,=axs[1].plot(res_ocean.Date,res_ocean.iloc[:,4],'.-',ms='12',label='April');
pp7,=axs[1].plot(res_ocean.Date,res_ocean.iloc[:,5],'^-',ms='8',label='May');
# axs[1].legend(title='Chl-a off shore',loc='upper right',ncol=3,bbox_to_anchor=(1.01,1.1))
axs[1].grid(ls='--')
axs[1].set_xticks(years);
axs[1].set_ylabel('Chl-a anomalies (mg m$^{-3}$)',y=1);
axs[1].text(0.005,0.88,'b)',weight='bold',transform=axs[1].transAxes)
axs[1].text(0.03,0.875,'Offshore',transform=axs[1].transAxes)
twin2=axs[1].twinx()
axs[1].set_xticks(years,rotation=45);
axs[1].set_xticklabels(years,rotation=45);

pp8,=twin2.plot(nao.Date[nao.Date<=2020],nao.NAO_DJF[nao.Date<=2020],'k',lw='0.5',label='NAO winter')
twin2.set_ylabel('NAO index')
twin2.grid(ls='dotted')

# pp9,=axs[2].plot(march_hplc_anom.Date.dt.year,march_hplc_anom.anomaly,'s-',ms='6',label='March');
# pp10,=axs[2].plot(april_hplc_anom.Date.dt.year,april_hplc_anom.anomaly,'.-',ms='12',label='April');
# pp11,=axs[2].plot(may_hplc_anom.Date.dt.year,may_hplc_anom.anomaly,'^-',ms='8',label='May');
# # axs[2].legend(title='Chl-a HPLC',ncol=3)
# axs[2].grid(ls='--')
# axs[2].set_xticks(march_hplc_anom.Date.dt.year,rotation=45);
# axs[2].set_xticklabels(years,rotation=45);
# axs[2].text(0.01,0.85,'c) HRTS',transform=axs[2].transAxes)
# twin3=axs[2].twinx()

# pp12,=twin3.plot(nao.Date[(nao.Date>=2004) & (nao.Date<=2020)],nao.NAO_DJF[(nao.Date>=2004) & (nao.Date<=2020)],'k',lw='0.5',label='NAO winter')
twin2.set_ylabel('NAO index',y=1)
twin2.grid(ls='dotted')


In [68]:
nao=nao[nao.Date<=2020]

In [69]:
# CORRELATION ANOMALIES NAO

print(sp.stats.pearsonr((res_ocean.iloc[8:11,3]),(nao.NAO_DJF[8:11]))[0]) # March
print(sp.stats.pearsonr((res_ocean.iloc[8:11,4]),(nao.NAO_DJF[8:11]))[0]) # April
print(sp.stats.pearsonr((res_ocean.iloc[8:11,5]),(nao.NAO_DJF[8:11]))[0]) # May

print(sp.stats.pearsonr((res_coast.iloc[8:11,3]),(nao.NAO_DJF[8:11]))[0]) # March
print(sp.stats.pearsonr((res_coast.iloc[8:11,4]),(nao.NAO_DJF[8:11]))[0]) # April
print(sp.stats.pearsonr((res_coast.iloc[8:11,5]),(nao.NAO_DJF[8:11]))[0]) # May

0.9987245028289352
0.9761108886515053
0.48183512326856215
0.9557204508006563
0.996060401985272
0.9408544676853097


In [73]:
fig,axs = plt.subplots(2, 1, figsize=(14,6),constrained_layout=True,sharex=True)
axs=axs.flatten()
axs[0].plot(res_coast.index,res_coast.iloc[:,8],'s-',ms='6',label='August');
axs[0].plot(res_coast.index,res_coast.iloc[:,9],'.-',ms='12',label='September');
axs[0].plot(res_coast.index,res_coast.iloc[:,10],'^-',ms='8',label='October');
axs[0].legend(ncol=3)
axs[0].grid(ls='--')
axs[0].set_xticks(res_ocean.index);
axs[0].text(-1,1.1,'a)',weight='bold')
axs[0].text(-0.2,1.1,'Coastal')

axs[1].plot(res_ocean.index,res_ocean.iloc[:,8],'s-',ms='6',label='August');
axs[1].plot(res_ocean.index,res_ocean.iloc[:,9],'.-',ms='12',label='September');
axs[1].plot(res_ocean.index,res_ocean.iloc[:,10],'^-',ms='8',label='October');
#axs[1].legend(title='Ocean',ncol=3)
axs[1].grid(ls='--')
axs[1].set_xticks(res_coast.index,years,rotation=45);
axs[1].set_ylabel('Chl-a anomalies (mg/m³)',y=1);
axs[1].text(-1,0.26,'b)',weight='bold')
axs[1].text(-0.2,0.26,'Ocean')

Text(-0.2, 0.26, 'Ocean')

In [74]:
print('August coast=',mk.seasonal_test(res_coast.iloc[:,8]))
print('September coast=',mk.seasonal_test(res_coast.iloc[:,9]))
print('October coast=',mk.seasonal_test(res_coast.iloc[:,10]))

print('August ocean=',mk.seasonal_test(res_ocean.iloc[:,8]))
print('September ocean=',mk.seasonal_test(res_ocean.iloc[:,9]))
print('October ocean=',mk.seasonal_test(res_ocean.iloc[:,10]))



August coast= Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=0.5464935954065819, z=0.6030226891555273, Tau=0.2727272727272727, s=3.0, var_s=11.0, slope=0.15023855865001678, intercept=-0.16004756403466064)
September coast= Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=1.0, z=0.0, Tau=-0.09090909090909091, s=-1.0, var_s=11.0, slope=-0.06846732646226883, intercept=0.09320016888280709)
October coast= Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=0.5464935954065819, z=-0.6030226891555273, Tau=-0.2727272727272727, s=-3.0, var_s=11.0, slope=-0.054622020572423935, intercept=0.05976745331039031)
August ocean= Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=1.0, z=0.0, Tau=0.09090909090909091, s=1.0, var_s=11.0, slope=0.03180208057165146, intercept=-0.03462375560775399)
September ocean= Seasonal_Mann_Kendall_Test(trend='no trend', h=False, p=1.0, z=0.0, Tau=0.09090909090909091, s=1.0, var_s=11.0, slope=0.02133457362651825, intercept=-0.03143155916283528)
Octo

# HISTOGRAMS

In [75]:
# ANOMALIES

res_coast_early=res_coast[res_coast.Date<2010].copy()
res_coast_late=res_coast[res_coast.Date>=2010].copy()
res_ocean_early=res_ocean[res_ocean.Date<2010].copy()
res_ocean_late=res_ocean[res_ocean.Date>=2010].copy()

# march_hplc_anom_early=march_hplc_anom[march_hplc_anom.Date.dt.year<2010].copy()
# april_hplc_anom_early=april_hplc_anom[april_hplc_anom.Date.dt.year<2010].copy()
# may_hplc_anom_early=may_hplc_anom[may_hplc_anom.Date.dt.year<2010].copy()

# march_hplc_anom_late=march_hplc_anom[march_hplc_anom.Date.dt.year>=2010].copy()
# april_hplc_anom_late=april_hplc_anom[april_hplc_anom.Date.dt.year>=2010].copy()
# may_hplc_anom_late=may_hplc_anom[may_hplc_anom.Date.dt.year>=2010].copy()



In [76]:
fig,axs=plt.subplots(3,2,constrained_layout=True,figsize=(12,9.5))

axs=axs.flatten()

sns.kdeplot(ax=axs[0],x=res_coast_early.iloc[:,3],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[0],x=res_coast_late.iloc[:,3],fill=True,lw=3,label='2010-2020')
axs[0].grid(ls='--')
axs[0].set_xlabel(None)
axs[0].set_ylabel(None)
axs[0].legend(title='Coast',ncol=2,bbox_to_anchor=(1,1.45))
axs[0].text(1.3,0.70,'March')
axs[0].text(0.01,0.87,'a)',weight='bold',transform=axs[0].transAxes)

sns.kdeplot(ax=axs[1],x=res_ocean_early.iloc[:,3],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[1],x=res_ocean_late.iloc[:,3],fill=True,lw=3,label='2010-2020')
axs[1].grid(ls='--')
axs[1].set_xlabel(None)
axs[1].set_ylabel(None)
axs[1].legend(title='Offshore',ncol=2,bbox_to_anchor=(1,1.45))
axs[1].text(0.01,0.87,'b)',weight='bold',transform=axs[1].transAxes)

sns.kdeplot(ax=axs[2],x=res_coast_early.iloc[:,4],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[2],x=res_coast_late.iloc[:,4],fill=True,lw=3,label='2010-2020')
axs[2].grid(ls='--')
axs[2].set_xlabel(None)
axs[2].set_ylabel('PDF')
# axs[2].legend(title='Coast April')
axs[2].text(3.6,0.42,'April')
axs[2].text(0.01,0.87,'c)',weight='bold',transform=axs[2].transAxes)

sns.kdeplot(ax=axs[3],x=res_ocean_early.iloc[:,4],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[3],x=res_ocean_late.iloc[:,4],fill=True,lw=3,label='2010-2020')
axs[3].grid(ls='--')
axs[3].set_xlabel(None)
axs[3].set_ylabel(None)
# axs[3].legend(title='Off shore April')
axs[3].text(0.01,0.87,'d)',weight='bold',transform=axs[3].transAxes)

sns.kdeplot(ax=axs[4],x=res_coast_early.iloc[:,5],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[4],x=res_coast_late.iloc[:,5],fill=True,lw=3,label='2010-2020')
axs[4].grid(ls='--')
axs[4].set_xlabel(None)
axs[4].set_ylabel(None)
# axs[4].legend(title='Coast April')
axs[4].text(3,0.6,'May')
axs[4].text(0.01,0.87,'e)',weight='bold',transform=axs[4].transAxes)

sns.kdeplot(ax=axs[5],x=res_ocean_early.iloc[:,5],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[5],x=res_ocean_late.iloc[:,5],fill=True,lw=3,label='2010-2020')
axs[5].grid(ls='--')
axs[5].set_xlabel(None)
axs[5].set_ylabel(None)
# axs[5].legend(title='Off shore April')
axs[5].text(0.01,0.87,'f)',weight='bold',transform=axs[5].transAxes)

plt.xlabel('Chl-a anomalies (mg m$^{-3}$)',x=-0.1)

Text(-0.1, 0, 'Chl-a anomalies (mg m$^{-3}$)')

In [78]:
# February

fig,axs=plt.subplots(1,2,constrained_layout=True,figsize=(12,5.5))

axs=axs.flatten()

sns.kdeplot(ax=axs[0],x=res_coast_early.iloc[:,2],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[0],x=res_coast_late.iloc[:,2],fill=True,lw=3,label='2010-2020')
axs[0].grid(ls='--')
axs[0].set_xlabel(None)
axs[0].set_ylabel('PDF')
axs[0].legend(title='Coast',ncol=2,bbox_to_anchor=(1,1.3))
axs[0].text(0.95,1.7,'February')
axs[0].text(0.01,0.87,'a)',weight='bold',transform=axs[0].transAxes)

sns.kdeplot(ax=axs[1],x=res_ocean_early.iloc[:,2],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[1],x=res_ocean_late.iloc[:,2],fill=True,lw=3,label='2010-2020')
axs[1].grid(ls='--')
axs[1].set_xlabel(None)
axs[1].set_ylabel(None)
axs[1].legend(title='Offshore',ncol=2,bbox_to_anchor=(1,1.3))
axs[1].text(0.01,0.87,'b)',weight='bold',transform=axs[1].transAxes)

plt.xlabel('Chl-a anomalies (mg m$^{-3}$)',x=-0.1)

Text(-0.1, 0, 'Chl-a anomalies (mg m$^{-3}$)')

In [79]:
fig,axs=plt.subplots(1,2,constrained_layout=True,figsize=(11,5))
sns.kdeplot(ax=axs[0],x=sat_coast.CHL[sat_coast.time.dt.year<2010],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[0],x=sat_coast.CHL[sat_coast.time.dt.year>=2010],fill=True,lw=3,label='2010-2020')
axs[0].grid(ls='--')
axs[0].set_xlabel(None)
axs[0].set_ylabel('PDF')
axs[0].legend(title='SAT Coast')

sns.kdeplot(ax=axs[1],x=sat_ocean.CHL[sat_ocean.time.dt.year<2010],fill=True,lw=3,label='1998-2009')
sns.kdeplot(ax=axs[1],x=sat_ocean.CHL[sat_ocean.time.dt.year>=2010],fill=True,lw=3,label='2010-2020')
axs[1].grid(ls='--')
axs[1].set_xlabel(None)
axs[1].set_ylabel(None)
axs[1].legend(title='SAT Off shore')

plt.xlabel('mg/m³',x=-0.1)

Text(-0.1, 0, 'mg/m³')

# MAXIMUM CHL MONTH

In [82]:
chl_clim = chl_clim.rename({'month': 'time'})

In [83]:
# CLIMATOLOGY

max_month_clim=chl_clim.idxmax(dim='time').to_dataframe().reset_index(drop=False)



In [84]:
pv_max_month_clim= pd.pivot_table(max_month_clim, index=max_month_clim.lat, columns=max_month_clim.lon,values='time')


In [85]:
import matplotlib.colors as clb

X, Y = np.meshgrid(pv_max_month_clim.columns,pv_max_month_clim.index)
cmap = clb.ListedColormap(['b','aquamarine','lightseagreen','silver','purple','yellow',
                           'greenyellow','darkolivegreen','darksalmon','magenta','maroon','r'])
bounds=np.arange(1,13)
ticks = [1,2,3,4,5,6,7,8,9,10,11,12]


In [86]:
xx,yy = np.meshgrid(ds_bat.lon, ds_bat.lat)

In [87]:
fig=plt.figure(figsize=(14,8),constrained_layout=True)
ax=plt.axes(projection=ccrs.Mercator(central_longitude=7.1))

img=plt.pcolormesh(X,Y,pv_max_month_clim.values,shading='auto', cmap=cmap,transform=ccrs.PlateCarree())
plt.clim([1,12])
cbar=plt.colorbar(orientation='horizontal', ticks = ticks,shrink=0.59,pad=0.01)
tick_locs = (bounds + 0.5)*(12-1)/12
cbar.set_ticks(tick_locs,shrink=0.59,pad=0.01)
cbar.set_ticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
CS=ax.contour(xx,yy,ds_bat,levels=[-30], colors='black',linestyles=['dashed'],transform=ccrs.PlateCarree())
#CS1=ax.contour(xx,yy,ds_bat,levels=[-10], colors='grey',linestyles=['dashed'],transform=ccrs.PlateCarree())
plt.xlabel('Longitude (°)')
plt.ylabel('Latitude (°)')
ax.add_feature(cfeature.LAND, facecolor='grey')
ax.coastlines()
ax.set_yticks([ 53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
ax.set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
ax.set_yticklabels([53, 54,55,56]) # -90, -60, -30
ax.set_xticklabels([3, 6, 9]) # 3, 6, 9

[Text(-456409.9122524203, 0, '3'),
 Text(-122451.43987259835, 0, '6'),
 Text(211507.03250722372, 0, '9')]

# CORRELATION

In [89]:
# Monthly climatology and anomaly\n",
chl_clim = ds_chl.groupby('time.month').mean(dim='time')
#chl_mean = ds_month.mean(dim='time')
#chl_std = ds_month.std(dim='time')
chl_anomalies = ds_chl.groupby('time.month') - chl_clim
#chl_anom_mean = chl_anomalies.mean(dim='time')
#chl_anom_std = chl_anomalies.std(dim='time')


In [90]:
sst_clim = ds_sst.groupby('time.month').mean(dim='time')
#sst_mean = ds_sst.mean(dim='time')
#sst_std = ds_sst.std(dim='time')
sst_anomalies = ds_sst.groupby('time.month') - sst_clim
sst_anom_mean = sst_anomalies.mean(dim='time')
#sst_anom_std = sst_anomalies.std(dim='time')


In [91]:
mld_clim = ds_mld.groupby('time.month').mean(dim='time')
#mld_mean = ds_mld.mean(dim='time')
#mld_std = ds_mld.std(dim='time')
mld_anomalies = ds_mld.groupby('time.month') - mld_clim
#mld_anom_mean = mld_anomalies.mean(dim='time')
#mld_anom_std = mld_anomalies.std(dim='time')

In [92]:
sst_time_mean=sst_anomalies.mean(dim=['lat','lon'])

In [93]:
sst_time_mean=sst_time_mean.to_dataframe().reset_index(drop=False)

In [94]:
chl_time_mean=chl_anomalies.mean(dim=['lat','lon'])

In [95]:
chl_time_mean=chl_time_mean.to_dataframe().reset_index(drop=False)

In [96]:
mld_time_mean=mld_anomalies.mean(dim=['lat','lon'])

In [97]:
mld_time_mean=mld_time_mean.to_dataframe().reset_index(drop=False)

In [98]:
# Correlation

result_cor=[]
p_value=[]
result_cor1=[]
p_value1=[]
result_cor2=[]
p_value2=[]
result_cor3=[]
p_value3=[]

for lat in range(0,len(chl_anomalies.lat)):
    for lon in range(0,len(chl_anomalies.lon)):
#        try:
        slope, intercept, r_value, pp, std_err = sp.stats.linregress(mld_anomalies[:,lat,lon],chl_anomalies[:,lat,lon]) # MLD
        slope1, intercept1, r_value1, pp1, std_err1 = sp.stats.linregress(sst_anomalies[:,lat,lon],chl_anomalies[:,lat,lon]) # SST
        slope2, intercept2, r_value2, pp2, std_err2 = sp.stats.linregress(mld_anomalies[0:-1,lat,lon],chl_anomalies[1:,lat,lon]) # MLD shift
        slope3, intercept3, r_value3, pp3, std_err3 = sp.stats.linregress(sst_anomalies[0:-1,lat,lon],chl_anomalies[1:,lat,lon]) # SST shift          
#        except ZeroDivisionError:
#            r_value = np.nan  # MLD
#            r_value1 = np.nan # SST
#            r_value2 = np.nan  # MLD shift
#            r_value3 = np.nan  # SST shift          
#            pp = np.nan  # MLD
#            pp1 = np.nan  # SST
#            pp2 = np.nan  # MLD shift
#            pp3 = np.nan  # SST shift            
        #print(lat,lon)
        result_cor.append((r_value, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon]))) # MLD
        p_value.append((pp, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon])))
        result_cor1.append((r_value1, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon]))) # SST
        p_value1.append((pp1, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon]))) 

        result_cor2.append((r_value2, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon]))) # MLD shift
        p_value2.append((pp2, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon])))
        result_cor3.append((r_value3, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon]))) # SST shift
        p_value3.append((pp3, float(chl_anomalies.lat[lat]), float(chl_anomalies.lon[lon]))) 


result_cor=pd.DataFrame(result_cor, columns=('correlation', 'lat', 'lon'))
p_value=pd.DataFrame(p_value, columns=('p_value', 'lat', 'lon'))

result_cor1=pd.DataFrame(result_cor1, columns=('correlation', 'lat', 'lon'))
p_value1=pd.DataFrame(p_value1, columns=('p_value', 'lat', 'lon'))

result_cor2=pd.DataFrame(result_cor2, columns=('correlation', 'lat', 'lon'))
p_value2=pd.DataFrame(p_value2, columns=('p_value', 'lat', 'lon'))

result_cor3=pd.DataFrame(result_cor3, columns=('correlation', 'lat', 'lon'))
p_value3=pd.DataFrame(p_value3, columns=('p_value', 'lat', 'lon'))


In [99]:
chl_mld_cor=pd.DataFrame(result_cor)
chl_mld_cor_pvalue=pd.DataFrame(p_value)

chl_sst_cor=pd.DataFrame(result_cor1)
chl_sst_cor_pvalue=pd.DataFrame(p_value1)

In [100]:
chl_mld_cor_lag1=pd.DataFrame(result_cor2)
chl_mld_cor_pvalue_lag1=pd.DataFrame(p_value2)

chl_sst_cor_lag1=pd.DataFrame(result_cor3)
chl_sst_cor_pvalue_lag1=pd.DataFrame(p_value3)

In [ ]:
chl_mld_cor=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/correlation_chl_mld.csv')
chl_mld_cor_pvalue=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/p_value_correlation_chl_mld.csv')

chl_sst_cor=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/correlation_chl_sst.csv')
chl_sst_cor_pvalue=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/p_value_correlation_chl_sst.csv')

In [ ]:
pv_chl_mld_cor= pd.pivot_table(chl_mld_cor, index=chl_mld_cor.lat, columns=chl_mld_cor.lon,values='correlation')

In [ ]:
pv_chl_sst_cor= pd.pivot_table(chl_sst_cor, index=chl_sst_cor.lat, columns=chl_sst_cor.lon,values='correlation')

In [ ]:
chl_mld_cor_lag1=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/correlation_chl_mld_1_lag.csv')
chl_mld_cor_pvalue_lag1=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/p_value_correlation_chl_mld_1_lag.csv')

chl_sst_cor_lag1=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/correlation_chl_sst_1_lag.csv')
chl_sst_cor_pvalue_lag1=pd.read_csv('/mnt/g/PhD/chapter_1/globcolour_daily/p_value_correlation_chl_sst_1_lag.csv')

In [ ]:
pv_chl_mld_cor_lag1= pd.pivot_table(chl_mld_cor_lag1, index=chl_mld_cor_lag1.lat, columns=chl_mld_cor_lag1.lon,values='correlation')

In [ ]:
pv_chl_sst_cor_lag1= pd.pivot_table(chl_sst_cor_lag1, index=chl_sst_cor_lag1.lat, columns=chl_sst_cor_lag1.lon,values='correlation')

In [ ]:
X1, Y1 = np.meshgrid(pv_chl_mld_cor.columns,pv_chl_mld_cor.index)

In [ ]:
X2, Y2 = np.meshgrid(pv_chl_sst_cor.columns,pv_chl_sst_cor.index)

In [ ]:
X3, Y3 = np.meshgrid(ds_bat_all.lon,ds_bat_all.lat)

In [ ]:
# Define the figure and each axis for the 1 rows and 2 columns
fig, axs = plt.subplots(nrows=2,ncols=2,
                        subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},
                        figsize=(13,9),sharey=True,sharex=True)
divnorm = mclr.TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=0.4)
# divnorm = mclr.TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)
# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

fig.subplots_adjust(bottom=0.1, top=0.98, left=0.08, right=0.99,
                    wspace=0.1, hspace=0.05)

img=axs[0].contourf(X2,Y2,pv_chl_sst_cor.values,norm=divnorm,cmap='seismic',
                                    levels=list(np.arange(-0.4,0.5,0.1)),transform=ccrs.PlateCarree())#
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_xlabel(None)
axs[0].set_ylabel('Latitude (°)',y=0)
axs[0].plot(chl_sst_cor_pvalue.lon[chl_sst_cor_pvalue.p_value<=0.05],
        chl_sst_cor_pvalue.lat[chl_sst_cor_pvalue.p_value<=0.05],
        'ko',transform=ccrs.PlateCarree(),markersize=1)
CS30=axs[0].contour(X3,Y3,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',lw=1,transform=ccrs.PlateCarree())
CS5=axs[0].contour(X2,Y2,pv_chl_sst_cor,levels=[0.5], colors='k',linestyles='solid',transform=ccrs.PlateCarree())
axs[0].add_feature(cfeature.LAND, facecolor='grey')
axs[0].coastlines()
axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[0].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[0].set_xticklabels([])# 3, 6, 9    
axs[0].text(2.4,56.05,'a)',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9 
axs[0].text(7.5,53.06,'Chl-a|SST',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9

img2=axs[1].contourf(X2,Y2,pv_chl_sst_cor_lag1.values,norm=divnorm,cmap='seismic',
                                    levels=list(np.arange(-0.4,0.5,0.1)),transform=ccrs.PlateCarree())#np.arange(-1,1.1,0.2)
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_xlabel(None)
axs[1].set_ylabel(None)
axs[1].plot(chl_sst_cor_pvalue_lag1.lon[chl_sst_cor_pvalue_lag1.p_value<=0.05],
        chl_sst_cor_pvalue_lag1.lat[chl_sst_cor_pvalue_lag1.p_value<=0.05],
        'ko',transform=ccrs.PlateCarree(),markersize=1)
CS30a=axs[1].contour(X3,Y3,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
axs[1].add_feature(cfeature.LAND, facecolor='grey')
axs[1].coastlines()
axs[1].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[1].set_yticklabels([]) # -90, -60, -30
axs[1].set_xticklabels([]) 
axs[1].set_ylabel(None)
axs[1].text(2.4,56.05,'b)',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9  
axs[1].text(7,53.06,'Chl-a|SST t-1',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9

img3=axs[2].contourf(X1,Y1,pv_chl_mld_cor.values,norm=divnorm,cmap='seismic',
                                    levels=list(np.arange(-0.4,0.5,0.1)),transform=ccrs.PlateCarree())#
axs[2].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[2].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[2].set_xlabel(None)
axs[2].set_ylabel(None)
axs[2].plot(chl_mld_cor_pvalue.lon[chl_mld_cor_pvalue.p_value<=0.05],
        chl_mld_cor_pvalue.lat[chl_mld_cor_pvalue.p_value<=0.05],
        'ko',transform=ccrs.PlateCarree(),markersize=1)
CS30=axs[2].contour(X3,Y3,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
CS5=axs[2].contour(X1,Y1,pv_chl_mld_cor,levels=[0.5], colors='k',linestyles='solid',transform=ccrs.PlateCarree())
axs[2].add_feature(cfeature.LAND, facecolor='grey')
axs[2].coastlines()
axs[2].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[2].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[2].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[2].set_xticklabels([])# 3, 6, 9    
axs[2].text(2.4,56.05,'c)',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9  
axs[2].text(7.5,53.06,'Chl-a|MLD',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9

img4=axs[3].contourf(X1,Y1,pv_chl_mld_cor_lag1.values,norm=divnorm,cmap='seismic',
                                    levels=list(np.arange(-0.4,0.5,0.1)),transform=ccrs.PlateCarree())#
axs[3].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30,
axs[3].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[3].set_xlabel('Longitude (°)',x=0)
axs[3].plot(chl_mld_cor_pvalue_lag1.lon[chl_mld_cor_pvalue_lag1.p_value<=0.05],
        chl_mld_cor_pvalue_lag1.lat[chl_mld_cor_pvalue_lag1.p_value<=0.05],
        'ko',transform=ccrs.PlateCarree(),markersize=1)
CS30a=axs[3].contour(X3,Y3,ds_bat_all,levels=[-30], colors='k',linestyles='dashed',transform=ccrs.PlateCarree())
axs[3].add_feature(cfeature.LAND, facecolor='grey')
axs[3].coastlines()
axs[3].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[3].set_xticks([3, 6, 9], crs=ccrs.PlateCarree())
axs[3].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[3].set_xticklabels([3, 6, 9]) 
axs[3].set_ylabel(None)
axs[3].text(2.4,56.05,'d)',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9  
axs[3].text(7.,53.06,'Chl-a|MLD t-1',weight='bold',transform=ccrs.PlateCarree())# 3, 6, 9

cbar=plt.colorbar(img4,orientation='vertical',shrink=0.96,pad=0.01,ticks=list(np.arange(-0.4,0.5,0.1)),ax=axs.ravel().tolist(),label='Correlation')

# EOF CHL ANOMALIES AND CLIMATOLOGY

In [126]:
# HIGHER RESOLUTION #####
# Monthly climatology and anomaly
chl_clim = ds_month.groupby('time.month').mean(dim='time')
chl_anomalies = ds_month.groupby('time.month') - chl_clim


In [127]:
chl_clim = chl_clim.rename({'month': 'time'})

In [103]:
mca = xMCA(chl_anomalies) # Chl-a anomalies high resolution
mca.normalize()
mca.apply_coslat()
mca.solve()

In [104]:
svals = mca.singular_values()

In [ ]:
plt.figure()
plt.plot(svals)

In [105]:
variance_x = mca.variance() 

In [ ]:
plt.figure()
plt.plot(variance_x)
plt.plot(svals)

In [106]:
expvar = mca.explained_variance()

In [ ]:
plt.figure(constrained_layout=True)
plt.plot(expvar.mode[0:10],expvar[0:10],'k')
plt.plot(expvar.mode[0:10],expvar[0:10],'ro')
plt.xlabel('Mode')
plt.xticks([1,2,3,4,5,6,7,8,9,10])
plt.grid(ls='--')

In [ ]:
plt.figure()
plt.plot(expvar,'-ok')
#plt.plot(variance_fractions.mode+1,variance_fractions*100,'or')
plt.ylabel('Explained variance (%)')
plt.grid(ls='--')

In [107]:
svals_err_north = mca.rule_north().to_dataframe()
svals_diff = svals.to_dataframe().diff(-1)
cutoff = np.argmax((svals_diff - (2 * svals_err_north)) < 0)  # 10

In [ ]:
cutoff

In [ ]:
del(eofs)
del(pcs)

In [108]:
eofs  = mca.eofs(scaling='max')
pcs   = mca.pcs(scaling='max')


In [109]:
eofs=eofs['left']
pcs=pcs['left']


In [112]:
# EOF CHL ANOMALIES MANUSCRIPT

levels=[-30]
# Define the figure and each axis for the 3 rows and 4 columns
fig, axs = plt.subplots(nrows=2,ncols=2,
                        subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},
                        figsize=(10,6.7),constrained_layout=True,sharey=True,sharex=True)

# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

fig.subplots_adjust(bottom=0.1, top=0.98, left=0.1, right=0.9,
                   wspace=0.01, hspace=0.00)
nn=0

for i in range(0,4):


    img=eofs.sel(mode=i+1).squeeze().plot.pcolormesh(
            ax=axs[i], cmap='coolwarm',add_colorbar=False, extend='both',vmin=-1, vmax=1,
            add_labels=False,transform=ccrs.PlateCarree())
    CS=axs[i].contour(xx,yy,eofs.sel(mode=i+1),levels=[0], colors='k',linewidth=1.5,transform=ccrs.PlateCarree())
    CS1=axs[i].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyle='dashed',linewidth=0.5,transform=ccrs.PlateCarree())    
    axs[i].set_ylabel(None)
    axs[i].set_xlabel(None)
    axs[i].add_feature(cfeature.LAND, facecolor='grey')
    axs[i].coastlines()
    frac = str(np.array(expvar[i]).round(2))
    axs[i].set_title('EOF'+str(i+1)+' ('+frac+'%)',fontsize=14)
cbar=plt.colorbar(img,orientation='vertical',pad=0.01,shrink=0.935,aspect=20,ticks=list(np.arange(-1,1.2,0.2)),ax=axs.ravel().tolist())

axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[0].set_ylabel('Latitude (°)',y=0)
axs[2].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[2].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[2].set_xticks([3, 6, 9], crs=ccrs.PlateCarree()) 
axs[2].set_xticklabels([3, 6, 9]) 
axs[3].set_xticks([3, 6, 9], crs=ccrs.PlateCarree()) 
axs[3].set_xticklabels([3, 6, 9])
axs[2].set_xlabel('Longitude (°)',x=1)
h1,_ = CS.legend_elements()
axs[3].legend([h1[0]],['0'],loc='lower right',bbox_to_anchor=(1.03,-0.02))

/tmp/ipykernel_1142/783541238.py:12: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.subplots_adjust(bottom=0.1, top=0.98, left=0.1, right=0.9,
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs were not used by contour: 'linestyle', 'linewidth'
  result = super().contour(*args, **kwargs)
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs

In [117]:
months=['J','F','M','A','M','J','J','A','S','O','N','D']
levels=[-30]

In [113]:
yearlist = np.unique(pcs.time.dt.year)

In [114]:
from matplotlib.dates import date2num
from datetime import datetime

In [115]:
letters=['a)','e)','i)','b)','f)','j)','c)','g)','k)','d)','h)','l)']

In [119]:
# PLOT PCs CHL ANOMALIES

fig,axs = plt.subplots(4, 3, figsize=(14,9),constrained_layout=True,sharex='col',gridspec_kw={'width_ratios': [2, 1, 1]})
axs=axs.flatten()
mm=1
for i in range(0,12,3):
#    pp1,=axs[0].plot(pcs.time,pcs.sel(mode=1))
#    axs[0].set_ylim([-1,1])
#    axs[1].plot(pcs.sel(mode=1).groupby('time.month').mean(dim='time').month,pcs.sel(mode=1).groupby('time.month').mean(dim='time'))
#    axs[1].set_xticks(pcs.sel(mode=1).groupby('time.month').mean(dim='time').month.values,
#                      months);
    pp1,=axs[i].plot(pcs.time,pcs.sel(mode=mm),'k',lw=1)
    pp2,=axs[i].plot(pcs.time,pcs.sel(mode=mm).rolling(time=6,center=True).mean(),'r',lw=1)
    axs[i].set_xlim(datetime(1998,1,1),datetime(2020,12,31))
    axs[i].grid(ls='--')
    frac = str(np.array(expvar.sel(mode=mm)).round(2))
    axs[i].set_ylabel('PC'+str(mm)+' ('+frac+'%)')            
    # yearlist = np.unique(pcs.time.dt.year)
    # for ii in range(len(yearlist)):
        
    #     axs[i].axvspan(date2num(datetime(int(yearlist[ii]),3,1)), date2num(datetime(int(yearlist[ii]),5,31))
    #                    ,color='green', alpha=0.25)
    #     axs[i].axvspan(date2num(datetime(int(yearlist[ii]),8,1)), date2num(datetime(int(yearlist[ii]),10,31))
    #                    ,color='brown', alpha=0.25)
    
#    axs[i].set_ylim([-0.5,0.5])
    # axs[i].set_ylim([-1,1])    
    pp3,=axs[i+1].plot(pcs.sel(mode=mm).groupby('time.month').mean(dim='time').month,pcs.sel(mode=mm).groupby('time.month').mean(dim='time'),'k')
    axs[i+1].set_xticks(pcs.sel(mode=mm).groupby('time.month').mean(dim='time').month.values,
                      months);
    axs[i+1].grid(ls='--')        
    axs[i+1].axvspan(3, 5,color='green', alpha=0.25)
    axs[i+1].axvspan(8, 10,color='brown', alpha=0.25)
    h,f,c=specs(pcs.sel(mode=mm),len(pcs.sel(mode=mm)),1,1,1,5,95)  
    axs[i+2].loglog(1/f,h,'k',label='PC'+str(mm))
    sa1=axs[i+2].axvline(x=12,color='r')
    sa2=axs[i+2].axvline(x=18,color='b',ls='-.')
    sa3=axs[i+2].axvline(x=6,color='r',ls='--')    
    axs[i+2].grid(ls='--')
        
    mm=mm+1
axs[2].legend([sa3,sa1,sa2],['6','12','18'],ncol=2,bbox_to_anchor=(0.98,1.6))
axs[11].set_xlabel('Months')
axs[8].set_ylabel('Spectral density',y=1)
axs[0].legend([pp1,pp2],['PC','Rolling mean 6 points'],loc='upper left',bbox_to_anchor=(0,1.6))
axs[1].legend([pp3],['PC mean'],loc='upper left',bbox_to_anchor=(0.25,1.4))

for ii in range(0,12):
    axs[ii].text(0.01,0.87,letters[ii],weight='bold',transform=axs[ii].transAxes)

In [137]:
del(mca)

In [138]:
mca = xMCA(chl_clim) # Chl-a climatology
mca.normalize()
mca.apply_coslat()
mca.solve()

In [139]:
svals = mca.singular_values()

In [140]:
variance_x = mca.variance() 

In [141]:
expvar = mca.explained_variance()

In [142]:
del(eofs)
del(pcs)

In [143]:
eofs  = mca.eofs(scaling='max')
pcs   = mca.pcs(scaling='max')

In [144]:
eofs=eofs['left']
pcs=pcs['left']

In [145]:
# EOF climatology

levels=[-30]
# Define the figure and each axis for the 3 rows and 4 columns
fig, axs = plt.subplots(nrows=1,ncols=2,
                        subplot_kw={'projection': ccrs.Mercator(central_longitude=7.1)},
                        figsize=(10,7),constrained_layout=True,sharey=True,sharex=True)

# axs is a 2 dimensional array of `GeoAxes`.  We will flatten it into a 1-D array
axs=axs.flatten()

fig.subplots_adjust(bottom=0.1, top=0.98, left=0.1, right=0.9,
                   wspace=0.01, hspace=0.00)
nn=0

for i in range(0,2):

    if i==0:
        
        img=(eofs.sel(mode=i+1)*-1).squeeze().plot.pcolormesh(
                ax=axs[i], vmin=-1, vmax=1, cmap='coolwarm',add_colorbar=False, extend='both',
                add_labels=False,transform=ccrs.PlateCarree())
        CS=axs[i].contour(xx,yy,eofs.sel(mode=i+1)*-1,levels=[0], colors='k',linewidth=1.5,transform=ccrs.PlateCarree())
        CS1=axs[i].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyle='dashed',linewidth=0.5,transform=ccrs.PlateCarree())    
        axs[i].set_ylabel(None)
        axs[i].set_xlabel(None)
        axs[i].add_feature(cfeature.LAND, facecolor='grey')
        axs[i].coastlines()
        frac = str(np.array(expvar[i]).round(2))
        axs[i].set_title('EOF'+str(i+1)+' ('+frac+'%)',fontsize=14)

    else:
        img=eofs.sel(mode=i+1).squeeze().plot.pcolormesh(
                ax=axs[i], vmin=-1, vmax=1, cmap='coolwarm',add_colorbar=False, extend='both',
                add_labels=False,transform=ccrs.PlateCarree())
        CS=axs[i].contour(xx,yy,eofs.sel(mode=i+1),levels=[0], colors='k',linewidth=1.5,transform=ccrs.PlateCarree())
        CS1=axs[i].contour(xx,yy,ds_bat,levels=[-30], colors='k',linestyle='dashed',linewidth=0.5,transform=ccrs.PlateCarree())    
        # h1,_ = CS1.legend_elements()
        # axs[i].legend([h1],['0'])
        axs[i].set_ylabel(None)
        axs[i].set_xlabel(None)
        axs[i].add_feature(cfeature.LAND, facecolor='grey')
        axs[i].coastlines()
        frac = str(np.array(expvar[i]).round(2))
        axs[i].set_title('EOF'+str(i+1)+' ('+frac+'%)',fontsize=14)
        
# cbar=plt.colorbar(img,orientation='vertical',pad=0.01,shrink=0.45,aspect=40,ticks=list(np.arange(-1,1.2,0.25)),ax=axs.ravel().tolist())
cbar=plt.colorbar(img,orientation='vertical',pad=0.01,shrink=0.41,aspect=10,ticks=list(np.arange(-1,1.2,0.25)),ax=axs.ravel().tolist())

axs[0].set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
axs[0].set_yticklabels([53, 54,55,56]) # -90, -60, -30
axs[0].set_ylabel('Latitude (°)')
axs[0].set_xticks([3, 6, 9], crs=ccrs.PlateCarree()) 
axs[0].set_xticklabels([3, 6, 9]) 
axs[1].set_xticks([3, 6, 9], crs=ccrs.PlateCarree()) 
axs[1].set_xticklabels([3, 6, 9]) 
axs[1].set_xlabel('Longitude (°)',x=0)
h1,_ = CS.legend_elements()
axs[0].legend([h1[0]],['0'],loc='lower right',bbox_to_anchor=(1.03,-0.04))


/tmp/ipykernel_1142/93394656.py:12: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.subplots_adjust(bottom=0.1, top=0.98, left=0.1, right=0.9,
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs were not used by contour: 'linestyle', 'linewidth'
  result = super().contour(*args, **kwargs)
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = super().contour(*args, **kwargs)
/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: The following kwargs 

In [134]:
# PC CLIMATOLOGY

fig,axs = plt.subplots(2, 1, figsize=(10,4.5),constrained_layout=True,sharex='col')
axs=axs.flatten()
mm=1
for i in range(0,2):

    if i==0:
    
        pp1,=axs[i].plot(pcs.time,pcs.sel(mode=mm)*-1,'k',lw=1)
        axs[i].grid(ls='--')
        frac = str(np.array(expvar.sel(mode=mm)).round(2))
        axs[i].set_ylabel('PC'+str(mm)+' ('+frac+'%)')            

        axs[i].axvspan(3, 5,color='green', alpha=0.25)
        axs[i].axvspan(8, 10,color='brown', alpha=0.25)
        axs[i].set_xticks(pcs.sel(mode=mm).time.values,
                          months);
        axs[i].set_xlim([1,12])
        axs[i].set_ylim([-1,1])
        twin1=axs[i].twinx()
        # pp,=twin1.plot(pcs.month.values,chl_clim_ocean,'k--',lw=0.5,label='mean')
        pp,=twin1.plot(chl_clim.time.values,chl_clim.mean(dim=('lat','lon')),'k--',lw=0.5,label='mean')        
        twin1.set_ylabel('Chl-a (mg m$^{-3}$)',y=0)
        twin1.grid(ls='dotted')
        axs[i].legend([pp1,pp],['PC','Chl-a mean (mg m$^{-3}$)'],ncol=2,loc='upper right',bbox_to_anchor=(1.,1.05))
        mm=mm+1

    else:

        pp1,=axs[i].plot(pcs.time,pcs.sel(mode=mm),'k',lw=1)
        axs[i].grid(ls='--')
        frac = str(np.array(expvar.sel(mode=mm)).round(2))
        axs[i].set_ylabel('PC'+str(mm)+' ('+frac+'%)')            

        axs[i].axvspan(3, 5,color='green', alpha=0.25)
        axs[i].axvspan(8, 10,color='brown', alpha=0.25)
        axs[i].set_xticks(pcs.sel(mode=mm).time.values,
                          months);
        axs[i].set_xlim([1,12])
        axs[i].set_ylim([-1,1])
        twin1=axs[i].twinx()
        # pp,=twin1.plot(chl_clim_ocean.month.values,chl_clim_ocean,'k--',lw=0.5,label='mean')
        pp,=twin1.plot(chl_clim.time.values,chl_clim.mean(dim=('lat','lon')),'k--',lw=0.5,label='mean') 
        twin1.grid(ls='dotted')
        mm=mm+1    


# MCA

In [157]:
# Monthly climatology and anomaly\n",
chl_clim = ds_chl.groupby('time.month').mean(dim='time')
#chl_mean = ds_month.mean(dim='time')
#chl_std = ds_month.std(dim='time')
chl_anomalies = ds_chl.groupby('time.month') - chl_clim
#chl_anom_mean = chl_anomalies.mean(dim='time')
#chl_anom_std = chl_anomalies.std(dim='time')


In [158]:
sst_clim = ds_sst.groupby('time.month').mean(dim='time')
#sst_mean = ds_sst.mean(dim='time')
#sst_std = ds_sst.std(dim='time')
sst_anomalies = ds_sst.groupby('time.month') - sst_clim
sst_anom_mean = sst_anomalies.mean(dim='time')
#sst_anom_std = sst_anomalies.std(dim='time')


In [159]:
mld_clim = ds_mld.groupby('time.month').mean(dim='time')
#mld_mean = ds_mld.mean(dim='time')
#mld_std = ds_mld.std(dim='time')
mld_anomalies = ds_mld.groupby('time.month') - mld_clim
#mld_anom_mean = mld_anomalies.mean(dim='time')
#mld_anom_std = mld_anomalies.std(dim='time')

In [160]:
del(mca)

In [161]:
mca = xMCA(chl_anomalies,sst_anomalies)
mca.normalize()
mca.apply_coslat()
mca.solve()

In [162]:
svals = mca.singular_values()

In [163]:
variance_x = mca.variance() 

In [164]:
expvar = mca.explained_variance()

In [165]:
svals_err_north = mca.rule_north().to_dataframe()
svals_diff = svals.to_dataframe().diff(-1)
cutoff = np.argmax((svals_diff - (2 * svals_err_north)) < 0)  # 10

In [166]:
del(eofs)
del(pcs)

In [167]:
eofs  = mca.eofs(scaling='max')
pcs   = mca.pcs(scaling='max')

In [168]:
eofs_ch=eofs['left']
pcs_ch=pcs['left']

eofs_sst=eofs['right']
pcs_sst=pcs['right']

In [169]:
mode=1
index=0

fig = plt.figure(figsize = (14, 8))

ax0 = plt.subplot2grid((2, 2), (0, 0),colspan=1, rowspan=1, projection=ccrs.Mercator(central_longitude=7.1))
pcm=ax0.pcolormesh(X3,Y3,-1*eofs_ch[:,:,index].squeeze(),shading='auto',vmin=-1,vmax=1 ,cmap='coolwarm',transform=ccrs.PlateCarree())
CS=ax0.contour(X3,Y3,-1*eofs_ch.sel(mode=mode),levels=[0], colors='k',transform=ccrs.PlateCarree())
CS1=ax0.contour(X3,Y3,ds_bat_all,levels=[-30], colors='grey',transform=ccrs.PlateCarree())   
fig.colorbar(pcm,ax=ax0,label='',pad=0.01)
ax0.add_feature(cfeature.LAND, facecolor='grey', edgecolor='k')
ax0.coastlines()
frac = str(np.array(expvar[index]).round(2))
ax0.set_title('Chl-a EOF'+str(mode)+' ('+frac+'%)',fontsize=14)
ax0.set_xticklabels([])
ax0.set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
ax0.set_yticklabels([53, 54,55,56]) # -90, -60, -30 
ax0.set_xticks([3, 6,9], crs=ccrs.PlateCarree()) # -90, -60, -30
ax0.set_xticklabels([3,6,9]) # -90, -60, -30 
ax0.set_ylabel('Latitude (°)')

ax1 = plt.subplot2grid((2, 2), (0, 1), colspan=1, rowspan=1, projection=ccrs.Mercator(central_longitude=7.1))
pcm=ax1.pcolormesh(X3,Y3,-1*eofs_sst[:,:,index].squeeze(),shading='auto',vmin=-1,vmax=1, cmap='coolwarm',transform=ccrs.PlateCarree())
CS=ax1.contour(X3,Y3,-1*eofs_sst.sel(mode=mode),levels=[0], colors='k',transform=ccrs.PlateCarree())
CS1=ax1.contour(X3,Y3,ds_bat_all,levels=[-30], colors='grey',transform=ccrs.PlateCarree()) 
fig.colorbar(pcm,ax=ax1,label='',pad=0.01)
ax1.add_feature(cfeature.LAND, facecolor='grey', edgecolor='k')
ax1.coastlines()
frac = str(np.array(expvar[index]).round(2))
ax1.set_title('SST EOF'+str(mode)+' ('+frac+'%)',fontsize=14)
ax1.set_xticklabels([])
ax1.set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
ax1.set_yticklabels([53, 54,55,56]) # -90, -60, -30 
ax1.set_xticks([3, 6,9], crs=ccrs.PlateCarree()) # -90, -60, -30
ax1.set_xticklabels([3,6,9]) # -90, -60, -30  

ax2 = plt.subplot2grid((2, 3), (1, 0), colspan=2, rowspan=1)
#ax2.plot(pcs_ch.time,-1*pcs_ch[:,index],'k',lw=1,label='Chl-a')
#ax2.plot(pcs_sst.time,-1*pcs_sst[:,index],'r',lw=1,label='SST')
ax2.plot(pcs_ch.time,-1*pcs_ch[:,index].rolling(time=6,center=True).mean(),'k',lw=1,label='Chl-a')
ax2.plot(pcs_sst.time,-1*pcs_sst[:,index].rolling(time=6,center=True).mean(),'r',lw=1,label='SST')
# yearlist = np.unique(pcs_sst.time.dt.year)
# for ii in range(len(yearlist)):
        
#     ax2.axvspan(date2num(datetime(int(yearlist[ii]),3,1)), date2num(datetime(int(yearlist[ii]),5,31))
#                     ,color='green', alpha=0.25)
#     ax2.axvspan(date2num(datetime(int(yearlist[ii]),8,1)), date2num(datetime(int(yearlist[ii]),10,31))
#                     ,color='brown', alpha=0.25)
frac = str(np.array(expvar[index]).round(2))
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(pcs_sst[:,index],pcs_ch[:,index])
ax2.set_ylabel('PC'+str(mode))
if p_value<=0.05:
    ax2.set_title('r='+str(round(r_value,2))+' p<'+str(0.05),fontsize=16)
else:
    ax2.set_title('r='+str(round(r_value,2))+' p>'+str(0.05),fontsize=16)
ax2.grid(ls='--')
ax2.set_xlim(['1998-01-31','2020-12-31'])
ax2.legend()
ax3 = plt.subplot2grid((2, 3), (1, 2), colspan=1, rowspan=2)
h,f,c=specs(pcs_ch.sel(mode=mode),len(pcs_ch.sel(mode=mode)),1,1,1,5,95)  
h1,f1,c1=specs(pcs_sst.sel(mode=mode),len(pcs_sst.sel(mode=mode)),1,1,1,5,95)  
ax3.loglog(1/f,h,'k',label='PC'+str(mode)+' Chl-a')
ax3.loglog(1/f1,h1,'k--',label='PC'+str(mode)+' SST')
ax3.set_xlabel('Months')
ax3.grid(ls='--')
ax3.legend()

/home/famorim/py_felipe/phd_py/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1614: UserWarning: No contour levels were found within the data range.
  result = super().contour(*args, **kwargs)


In [171]:
del(mca)

In [172]:
mca = xMCA(chl_anomalies,mld_anomalies)
mca.normalize()
mca.apply_coslat()
mca.solve()

In [173]:
svals = mca.singular_values()

In [174]:
variance_x = mca.variance() 

In [175]:
expvar = mca.explained_variance()

In [176]:
svals_err_north = mca.rule_north().to_dataframe()
svals_diff = svals.to_dataframe().diff(-1)
cutoff = np.argmax((svals_diff - (2 * svals_err_north)) < 0)  # 10

In [177]:
del(eofs)
del(pcs)

In [178]:
eofs  = mca.eofs(scaling='max')
pcs   = mca.pcs(scaling='max')

In [179]:
del(eofs_ch,pcs_ch)
del(eofs_sst,pcs_sst)

In [180]:
eofs_ch=eofs['left']
pcs_ch=pcs['left']

eofs_mld=eofs['right']
pcs_mld=pcs['right']

In [182]:
mode=1
index=0

fig = plt.figure(figsize = (14, 8))

ax0 = plt.subplot2grid((2, 2), (0, 0),colspan=1, rowspan=1, projection=ccrs.Mercator(central_longitude=7.1))
pcm=ax0.pcolormesh(X3,Y3,-1*eofs_ch[:,:,index].squeeze(),shading='auto',vmin=-1,vmax=1 ,cmap='coolwarm',transform=ccrs.PlateCarree())
CS=ax0.contour(X3,Y3,-1*eofs_ch.sel(mode=mode),levels=[0], colors='k',transform=ccrs.PlateCarree())
CS1=ax0.contour(X3,Y3,ds_bat_all,levels=[-30], colors='grey',transform=ccrs.PlateCarree())   
fig.colorbar(pcm,ax=ax0,label='',pad=0.01)
ax0.add_feature(cfeature.LAND, facecolor='grey', edgecolor='k')
ax0.coastlines()
frac = str(np.array(expvar[index]).round(2))
ax0.set_title('Chl-a EOF'+str(mode)+' ('+frac+'%)',fontsize=14)
ax0.set_xticklabels([])
ax0.set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
ax0.set_yticklabels([53, 54,55,56]) # -90, -60, -30 
ax0.set_xticks([3, 6,9], crs=ccrs.PlateCarree()) # -90, -60, -30
ax0.set_xticklabels([3,6,9]) # -90, -60, -30 
ax0.set_ylabel('Latitude (°)')

ax1 = plt.subplot2grid((2, 2), (0, 1), colspan=1, rowspan=1, projection=ccrs.Mercator(central_longitude=7.1))
pcm=ax1.pcolormesh(X3,Y3,-1*eofs_mld[:,:,index].squeeze(),shading='auto',vmin=-1,vmax=1, cmap='coolwarm',transform=ccrs.PlateCarree())
CS=ax1.contour(X3,Y3,-1*eofs_mld.sel(mode=mode),levels=[0], colors='k',transform=ccrs.PlateCarree())
CS1=ax1.contour(X3,Y3,ds_bat_all,levels=[-30], colors='grey',transform=ccrs.PlateCarree()) 
fig.colorbar(pcm,ax=ax1,label='',pad=0.01)
ax1.add_feature(cfeature.LAND, facecolor='grey', edgecolor='k')
ax1.coastlines()
frac = str(np.array(expvar[index]).round(2))
ax1.set_title('MLD EOF'+str(mode)+' ('+frac+'%)',fontsize=14)
ax1.set_xticklabels([])
ax1.set_yticks([53, 54,55,56], crs=ccrs.PlateCarree()) # -90, -60, -30
ax1.set_yticklabels([53, 54,55,56]) # -90, -60, -30 
ax1.set_xticks([3, 6,9], crs=ccrs.PlateCarree()) # -90, -60, -30
ax1.set_xticklabels([3,6,9]) # -90, -60, -30  

ax2 = plt.subplot2grid((2, 3), (1, 0), colspan=2, rowspan=1)
#ax2.plot(pcs_ch.time,-1*pcs_ch[:,index],'k',lw=1,label='Chl-a')
#ax2.plot(pcs_mld.time,-1*pcs_mld[:,index],'r',lw=1,label='mld')
ax2.plot(pcs_ch.time,-1*pcs_ch[:,index].rolling(time=6,center=True).mean(),'k',lw=1,label='Chl-a')
ax2.plot(pcs_mld.time,-1*pcs_mld[:,index].rolling(time=6,center=True).mean(),'r',lw=1,label='MLD')
# yearlist = np.unique(pcs_mld.time.dt.year)
# for ii in range(len(yearlist)):
        
#     ax2.axvspan(date2num(datetime(int(yearlist[ii]),3,1)), date2num(datetime(int(yearlist[ii]),5,31))
#                     ,color='green', alpha=0.25)
#     ax2.axvspan(date2num(datetime(int(yearlist[ii]),8,1)), date2num(datetime(int(yearlist[ii]),10,31))
#                     ,color='brown', alpha=0.25)
frac = str(np.array(expvar[index]).round(2))
slope, intercept, r_value, p_value, std_err = sp.stats.linregress(pcs_mld[:,index],pcs_ch[:,index])
ax2.set_ylabel('PC'+str(mode))
if p_value<=0.05:
    ax2.set_title('r='+str(round(r_value,2))+' p<'+str(0.05),fontsize=16)
else:
    ax2.set_title('r='+str(round(r_value,2))+' p>'+str(0.05),fontsize=16)
ax2.grid(ls='--')
ax2.set_xlim(['1998-01-31','2020-12-31'])
ax2.legend()
ax3 = plt.subplot2grid((2, 3), (1, 2), colspan=1, rowspan=2)
h,f,c=specs(pcs_ch.sel(mode=mode),len(pcs_ch.sel(mode=mode)),1,1,1,5,95)  
h1,f1,c1=specs(pcs_mld.sel(mode=mode),len(pcs_mld.sel(mode=mode)),1,1,1,5,95)  
ax3.loglog(1/f,h,'k',label='PC'+str(mode)+' Chl-a')
ax3.loglog(1/f1,h1,'k--',label='PC'+str(mode)+' MLD')
ax3.set_xlabel('Months')
ax3.grid(ls='--')
ax3.legend()